<a href="https://colab.research.google.com/github/tristanhowells/MNIST/blob/main/V43_Green_Up_Complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### CELL 1 - GOOGLE DRIVE SETUP ###

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### CELL 2 - INSTALL DEPENDENCIES ###

!pip install stable-baselines3[extra] gymnasium shimmy pandas numpy pyarrow -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 4.0 MB/s eta 0:00:00


In [ ]:
### CELL 3 - CONFIGURATION (V43 - FULL FEATURE SET) ###

# V43: All Features + Green-up Strategy
# 755-dimensional observation space (vs V42's 323)

print("="*60)
print("V43 Configuration - Full Feature Set + Green-up")
print("="*60)

# ============================================================
# PATHS
# ============================================================
BASE_PATH = '/content/drive/MyDrive/Betfair_RL/V43_Full_Features'
DATA_DIR = '/content/drive/MyDrive/race_out'

import os
os.makedirs(BASE_PATH, exist_ok=True)

# ============================================================
# HYPERPARAMETERS - SAC
# ============================================================
SAC_LEARNING_RATE = 3e-4
SAC_BUFFER_SIZE = 100000
SAC_LEARNING_STARTS = 1000
SAC_BATCH_SIZE = 256
SAC_TAU = 0.005
SAC_GAMMA = 0.99
SAC_TRAIN_FREQ = 1
SAC_GRADIENT_STEPS = 1
SAC_ENT_COEF = 'auto'

# ============================================================
# ENVIRONMENT PARAMETERS
# ============================================================
MAX_CAPITAL = 1000.0
COMMISSION_RATE = 0.02  # 2% (training rate)

# Curriculum Learning
CURRICULUM_TOTAL_STEPS = 200000
CURRICULUM_WARMUP_STEPS = 20000

# V43: Graduated constraints (same as V42)
INITIAL_MIN_LIABILITY = 0.05      # Start very permissive
PRODUCTION_MIN_LIABILITY = 5.0     # End strict
INITIAL_ACTION_THRESHOLD = 0.01    # Start very permissive
PRODUCTION_ACTION_THRESHOLD = 0.3  # End strict

# Capital Management
RESERVE_RATIO = 0.20                # 20% capital reserve
MAX_EXPOSURE_MULTIPLIER = 1.5       # Can use 150% of available (with reserve)

# Rewards (same as V42)
STEP_REWARD_TRADE = 0.01
STEP_REWARD_NO_TRADE = -0.001
MTM_REWARD_SCALE = 5.0
SHARPE_REWARD_SCALE = 1.0
CAPITAL_PRESERVATION_BONUS = 0.1

# V43: New - Depth and volatility constraints
MIN_DEPTH_RATIO = 0.5              # Trade must be <50% of available depth
HIGH_VOLATILITY_THRESHOLD = 0.05   # ret_std_5s threshold
STALE_MARKET_THRESHOLD = 60        # secs_since_last_trade threshold

print(f"\n✅ Configuration loaded for V43")
print(f"\n📊 Feature Set:")
print(f"   Observation dimensions: 755 (vs V42's 323)")
print(f"   Runner features: 31 (vs V42's 13)")
print(f"   Unused features from V42: 18")
print(f"\n🎯 New Features:")
print(f"   • Order book depth (levels 2-3): 8 features")
print(f"   • Pre-calculated volatility: 2 features")
print(f"   • Pre-calculated ob_imbalance: 1 feature")
print(f"   • Pre-calculated rel_spread: 1 feature")
print(f"   • Direct prob_implied: 1 feature")
print(f"   • Trading activity: 2 features")
print(f"   • Data quality: 3 features")
print(f"   • Engineered depth metrics: 3 features")
print(f"\n💰 Capital: ${MAX_CAPITAL:.0f}")
print(f"   Commission: {COMMISSION_RATE*100}%")
print(f"   Reserve ratio: {RESERVE_RATIO*100}%")
print(f"\n🎓 Curriculum: {CURRICULUM_WARMUP_STEPS:,} warmup → {CURRICULUM_TOTAL_STEPS:,} total")
print(f"   Min liability: ${INITIAL_MIN_LIABILITY} → ${PRODUCTION_MIN_LIABILITY}")
print(f"   Action threshold: {INITIAL_ACTION_THRESHOLD} → {PRODUCTION_ACTION_THRESHOLD}")
print(f"\n🛡️  Safety Constraints:")
print(f"   Min depth ratio: {MIN_DEPTH_RATIO*100}%")
print(f"   High volatility threshold: {HIGH_VOLATILITY_THRESHOLD}")
print(f"   Stale market threshold: {STALE_MARKET_THRESHOLD}s")
print("="*60)

V43 Configuration - Full Feature Set + Green-up

✅ Configuration loaded for V43

📊 Feature Set:
   Observation dimensions: 755 (vs V42's 323)
   Runner features: 31 (vs V42's 13)
   Unused features from V42: 18

🎯 New Features:
   • Order book depth (levels 2-3): 8 features
   • Pre-calculated volatility: 2 features
   • Pre-calculated ob_imbalance: 1 feature
   • Pre-calculated rel_spread: 1 feature
   • Direct prob_implied: 1 feature
   • Trading activity: 2 features
   • Data quality: 3 features
   • Engineered depth metrics: 3 features

💰 Capital: $1000
   Commission: 2.0%
   Reserve ratio: 20.0%

🎓 Curriculum: 20,000 warmup → 200,000 total
   Min liability: $0.05 → $5.0
   Action threshold: 0.01 → 0.3

🛡️  Safety Constraints:
   Min depth ratio: 50.0%
   High volatility threshold: 0.05
   Stale market threshold: 60s


In [ ]:
### CELL 4 - ENVIRONMENT & TRAINING COMPONENTS (V43 - FULL FEATURES) ###

# V43 - Full Feature Set (755 dims) + Green-up Strategy
# All 26 available features per runner
# Depth checking, volatility-based sizing, quality filtering

import random
from collections import defaultdict, deque
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import BaseCallback, CallbackList
from stable_baselines3.common.monitor import Monitor
import warnings
import signal
import os

class FileLoadTimeout(Exception):
    """Raised when file loading times out."""
    pass

def timeout_handler(signum, frame):
    raise FileLoadTimeout("File loading timed out")

warnings.filterwarnings('ignore')

print("="*60)
print("Model Version: V43_Full_Features_Green_Up")
print("Algorithm: SAC + ALL Features + Green-up")
print("\n✨ Features:")
print("   • ALL 26 available features (100% utilization)")
print("   • 755-dimensional observation space")
print("   • Order book depth (levels 1-3)")
print("   • Pre-calculated volatility (ret_std_5s, ret_std_20s)")
print("   • Pre-calculated market metrics")
print("   • Trading activity signals")
print("   • Data quality indicators")
print("   • Green-up at in-play transition")
print("   • Depth-based trade filtering")
print("   • Volatility-based position sizing")
print("   • Stale market detection")
print("="*60)

# ============================================================
# TYPE-SAFE HELPERS
# ============================================================

def to_float(val, default=0.0):
    """Safely convert to float."""
    if val is None or val == '' or (isinstance(val, float) and np.isnan(val)):
        return default
    try:
        return float(val)
    except (ValueError, TypeError):
        return default

def safe_normalize(value, min_val, max_val, epsilon=1e-8):
    """Normalize to [0,1] with safety checks."""
    if max_val - min_val < epsilon:
        return 0.5
    normalized = (value - min_val) / (max_val - min_val)
    return np.clip(normalized, 0.0, 1.0)

def safe_log_norm(value, epsilon=1e-8):
    """Log-normalize with safety."""
    return np.log1p(max(0, value) + epsilon)

# ============================================================
# RUNNER DATA EXTRACTION (ALL 26 FEATURES!)
# ============================================================

def get_runner_data(row, runner_idx):
    """
    V43: Extract ALL available features for a runner.
    Returns 26 raw features per runner.
    """
    prefix = f'run[{runner_idx}].'

    try:
        # === LEVEL 1: PRICES ===
        back_1 = row.get(f'{prefix}back_price_1', None)
        lay_1 = row.get(f'{prefix}lay_price_1', None)

        if back_1 is None or lay_1 is None or pd.isna(back_1) or pd.isna(lay_1):
            return None

        # === LEVEL 1: VOLUMES ===
        back_vol_1 = row.get(f'{prefix}back_size_1', 0.0)
        lay_vol_1 = row.get(f'{prefix}lay_size_1', 0.0)

        if pd.isna(back_vol_1):
            back_vol_1 = 0.0
        if pd.isna(lay_vol_1):
            lay_vol_1 = 0.0

        # === LEVEL 2: PRICES & VOLUMES ===
        back_2 = row.get(f'{prefix}back_price_2', back_1)
        lay_2 = row.get(f'{prefix}lay_price_2', lay_1)
        back_vol_2 = row.get(f'{prefix}back_size_2', 0.0)
        lay_vol_2 = row.get(f'{prefix}lay_size_2', 0.0)

        if pd.isna(back_2): back_2 = back_1
        if pd.isna(lay_2): lay_2 = lay_1
        if pd.isna(back_vol_2): back_vol_2 = 0.0
        if pd.isna(lay_vol_2): lay_vol_2 = 0.0

        # === LEVEL 3: PRICES & VOLUMES ===
        back_3 = row.get(f'{prefix}back_price_3', back_1)
        lay_3 = row.get(f'{prefix}lay_price_3', lay_1)
        back_vol_3 = row.get(f'{prefix}back_size_3', 0.0)
        lay_vol_3 = row.get(f'{prefix}lay_size_3', 0.0)

        if pd.isna(back_3): back_3 = back_1
        if pd.isna(lay_3): lay_3 = lay_1
        if pd.isna(back_vol_3): back_vol_3 = 0.0
        if pd.isna(lay_vol_3): lay_vol_3 = 0.0

        # === DATA QUALITY INDICATORS ===
        has_level_1 = row.get(f'{prefix}has_level_1', True)
        has_level_2 = row.get(f'{prefix}has_level_2', False)
        has_level_3 = row.get(f'{prefix}has_level_3', False)

        if pd.isna(has_level_1): has_level_1 = True
        if pd.isna(has_level_2): has_level_2 = False
        if pd.isna(has_level_3): has_level_3 = False

        # === LAST TRADED ===
        ltp = row.get(f'{prefix}last_traded_price', back_1)
        if pd.isna(ltp):
            ltp = back_1

        # === TRADED VOLUME ===
        traded_vol_total = row.get(f'{prefix}traded_vol_total', 0.0)
        if pd.isna(traded_vol_total):
            traded_vol_total = 0.0

        traded_vol_60s = row.get(f'{prefix}traded_vol_60s', 0.0)
        if pd.isna(traded_vol_60s):
            traded_vol_60s = 0.0

        # === TIME SINCE LAST TRADE ===
        secs_since_last_trade = row.get(f'{prefix}secs_since_last_trade', 999.0)
        if pd.isna(secs_since_last_trade):
            secs_since_last_trade = 999.0

        # === PRE-CALCULATED FEATURES ===
        microprice = row.get(f'{prefix}microprice', None)
        if microprice is None or pd.isna(microprice):
            microprice = (back_1 + lay_1) / 2.0

        ob_imbalance = row.get(f'{prefix}ob_imbalance', None)
        if ob_imbalance is None or pd.isna(ob_imbalance):
            # Fallback: calculate simple level-1 imbalance
            total_vol = back_vol_1 + lay_vol_1
            if total_vol > 0:
                ob_imbalance = (back_vol_1 - lay_vol_1) / total_vol
            else:
                ob_imbalance = 0.0

        rel_spread = row.get(f'{prefix}rel_spread', None)
        if rel_spread is None or pd.isna(rel_spread):
            # Fallback: calculate
            spread = abs(back_1 - lay_1)
            rel_spread = spread / max(microprice, 1.01)

        prob_implied = row.get(f'{prefix}prob_implied', None)
        if prob_implied is None or pd.isna(prob_implied):
            # Fallback: use WOM
            total_vol = back_vol_1 + lay_vol_1
            if total_vol > 0:
                prob_implied = back_vol_1 / total_vol
            else:
                prob_implied = 1.0 / microprice if microprice > 1.01 else 0.5

        # === VOLATILITY (PRE-CALCULATED) ===
        ret_std_5s = row.get(f'{prefix}ret_std_5s', 0.0)
        if pd.isna(ret_std_5s):
            ret_std_5s = 0.0

        ret_std_20s = row.get(f'{prefix}ret_std_20s', 0.0)
        if pd.isna(ret_std_20s):
            ret_std_20s = 0.0

        # === RETURN ALL 26 FEATURES ===
        return {
            # Level 1 (4)
            'back_1': float(back_1),
            'lay_1': float(lay_1),
            'back_vol_1': float(back_vol_1),
            'lay_vol_1': float(lay_vol_1),

            # Level 2 (4)
            'back_2': float(back_2),
            'lay_2': float(lay_2),
            'back_vol_2': float(back_vol_2),
            'lay_vol_2': float(lay_vol_2),

            # Level 3 (4)
            'back_3': float(back_3),
            'lay_3': float(lay_3),
            'back_vol_3': float(back_vol_3),
            'lay_vol_3': float(lay_vol_3),

            # Data quality (3)
            'has_level_1': bool(has_level_1),
            'has_level_2': bool(has_level_2),
            'has_level_3': bool(has_level_3),

            # Trading activity (4)
            'ltp': float(ltp),
            'traded_vol_total': float(traded_vol_total),
            'traded_vol_60s': float(traded_vol_60s),
            'secs_since_last_trade': float(secs_since_last_trade),

            # Pre-calculated features (5)
            'microprice': float(microprice),
            'ob_imbalance': float(ob_imbalance),
            'rel_spread': float(rel_spread),
            'prob_implied': float(prob_implied),

            # Volatility (2)
            'ret_std_5s': float(ret_std_5s),
            'ret_std_20s': float(ret_std_20s),
        }

    except Exception as e:
        return None

# ============================================================
# CURRICULUM TRACKER (same as V42)
# ============================================================

class CurriculumTracker:
    """Graduated MIN_LIABILITY and ACTION_THRESHOLD."""

    def __init__(self, total_steps, warmup_steps):
        self.total_steps = total_steps
        self.warmup_steps = warmup_steps
        self.current_step = 0

        self.min_liability_start = INITIAL_MIN_LIABILITY
        self.min_liability_end = PRODUCTION_MIN_LIABILITY

        self.threshold_start = INITIAL_ACTION_THRESHOLD
        self.threshold_end = PRODUCTION_ACTION_THRESHOLD

    def step(self):
        self.current_step += 1

    def get_progress(self):
        """Quadratic ease-in curve."""
        if self.current_step < self.warmup_steps:
            return 0.0

        effective_step = self.current_step - self.warmup_steps
        effective_total = self.total_steps - self.warmup_steps

        if effective_step >= effective_total:
            return 1.0

        linear_progress = effective_step / effective_total
        return linear_progress ** 2

    def get_current_min_liability(self):
        progress = self.get_progress()
        return self.min_liability_start + (self.min_liability_end - self.min_liability_start) * progress

    def get_current_action_threshold(self):
        progress = self.get_progress()
        return self.threshold_start + (self.threshold_end - self.threshold_start) * progress

    def get_status_string(self):
        progress = self.get_progress() * 100
        min_lia = self.get_current_min_liability()
        threshold = self.get_current_action_threshold()
        return f"Curriculum: {progress:.0f}% | MIN_LIA=${min_lia:.2f} | θ={threshold:.3f}"

# ============================================================
# MARKET MAKING ENVIRONMENT (V43 - ALL FEATURES)
# ============================================================

class MarketMakingEnv(gym.Env):
    """V43: Full feature set (755 dims) + Green-up strategy."""

    metadata = {'render_modes': ['human']}

    def __init__(self, race_files, curriculum_tracker=None):
        super().__init__()

        print(f"\n[ENV INIT] V43 with ALL features (755 dimensions)")

        self.race_files = race_files
        self.curriculum_tracker = curriculum_tracker

        if self.curriculum_tracker:
            print(f"[ENV] Curriculum tracker attached ✅")
        else:
            print(f"[ENV] No curriculum (fixed constraints)")

        # Action: 24 runner signals + 1 allocation
        self.action_space = spaces.Box(
            low=-1.0, high=1.0, shape=(25,), dtype=np.float32
        )

        # V43: 755-dimensional observation space
        # 24 runners × 31 features + 11 global = 755
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(755,), dtype=np.float32
        )

        # Episode state
        self.current_race_df = None
        self.runner_count = 0
        self.step_idx = 0

        # Financial state
        self.balance = MAX_CAPITAL
        self.initial_balance = MAX_CAPITAL
        self.positions = {}
        self.total_commission_paid = 0.0
        self.total_mtm_reward = 0.0
        self.total_sharpe_reward = 0.0

        # Tracking
        self.trades_this_episode = []
        self.trades_last_10_steps = deque(maxlen=10)
        self.price_history = deque(maxlen=20)

        # MTM and Sharpe tracking
        self.previous_mtm_pnl = 0.0
        self.pnl_history = deque(maxlen=100)
        self.peak_balance = MAX_CAPITAL

        # Episode counter
        self.episode_number = 0

        # V43: Track depth violations for debugging
        self.depth_violations = 0
        self.volatility_violations = 0
        self.stale_market_violations = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        self.episode_number += 1

        # Load random race
        max_attempts = 5
        for attempt in range(max_attempts):
            try:
                race_file = random.choice(self.race_files)
                self.current_race_df = pd.read_parquet(race_file)
                break
            except Exception as e:
                if attempt == max_attempts - 1:
                    raise RuntimeError(f"Failed after {max_attempts} attempts")
                try:
                    if race_file in self.race_files:
                        self.race_files.remove(race_file)
                        print(f"⚠️  Skipping corrupt file")
                except:
                    pass

        self.runner_count = int(to_float(self.current_race_df.iloc[0]['runner_count'], 9))
        self.step_idx = 0
        self.positions = {}
        self.balance = MAX_CAPITAL
        self.initial_balance = MAX_CAPITAL
        self.total_commission_paid = 0.0
        self.trades_this_episode = []
        self.trades_last_10_steps.clear()
        self.price_history.clear()

        # Reset MTM tracking
        self.previous_mtm_pnl = 0.0
        self.pnl_history.clear()
        self.peak_balance = MAX_CAPITAL
        self.total_mtm_reward = 0.0
        self.total_sharpe_reward = 0.0

        # V43: Reset violation counters
        self.depth_violations = 0
        self.volatility_violations = 0
        self.stale_market_violations = 0

        return self._get_observation(), {}

    def _get_observation(self):
        """
        V43: Build 755-dimensional observation with ALL features.
        24 runners × 31 features + 11 global = 755 dims
        """
        if self.step_idx >= len(self.current_race_df):
            return np.zeros(755, dtype=np.float32)

        row = self.current_race_df.iloc[self.step_idx]
        obs = []

        # RUNNER OBSERVATIONS (31 features × 24 runners = 744)
        for runner_idx in range(24):
            if runner_idx < self.runner_count:
                runner_data = get_runner_data(row, runner_idx)

                if runner_data:
                    # === LEVEL 1: PRICES & VOLUMES (4 features) ===
                    back_1 = safe_normalize(runner_data['back_1'], 1.01, 1000.0)
                    lay_1 = safe_normalize(runner_data['lay_1'], 1.01, 1000.0)
                    back_vol_1 = safe_log_norm(runner_data['back_vol_1'])
                    lay_vol_1 = safe_log_norm(runner_data['lay_vol_1'])

                    # === LEVEL 2: PRICES & VOLUMES (4 features) ===
                    back_2 = safe_normalize(runner_data['back_2'], 1.01, 1000.0)
                    lay_2 = safe_normalize(runner_data['lay_2'], 1.01, 1000.0)
                    back_vol_2 = safe_log_norm(runner_data['back_vol_2'])
                    lay_vol_2 = safe_log_norm(runner_data['lay_vol_2'])

                    # === LEVEL 3: PRICES & VOLUMES (4 features) ===
                    back_3 = safe_normalize(runner_data['back_3'], 1.01, 1000.0)
                    lay_3 = safe_normalize(runner_data['lay_3'], 1.01, 1000.0)
                    back_vol_3 = safe_log_norm(runner_data['back_vol_3'])
                    lay_vol_3 = safe_log_norm(runner_data['lay_vol_3'])

                    # === PRE-CALCULATED FEATURES (6 features) ===
                    microprice = safe_normalize(runner_data['microprice'], 1.01, 1000.0)
                    ob_imbalance = safe_normalize(runner_data['ob_imbalance'], -1.0, 1.0)
                    rel_spread = safe_normalize(runner_data['rel_spread'], 0.0, 0.1)
                    prob_implied = safe_normalize(runner_data['prob_implied'], 0.0, 1.0)
                    ret_std_5s = safe_log_norm(runner_data['ret_std_5s'])
                    ret_std_20s = safe_log_norm(runner_data['ret_std_20s'])

                    # === TRADING ACTIVITY (3 features) ===
                    ltp = safe_normalize(runner_data['ltp'], 1.01, 1000.0)
                    traded_vol_total = safe_log_norm(runner_data['traded_vol_total'])
                    traded_vol_60s = safe_log_norm(runner_data['traded_vol_60s'])
                    secs_since_last_trade = safe_log_norm(runner_data['secs_since_last_trade'] + 1)

                    # === DATA QUALITY (3 features) ===
                    has_level_1_f = float(runner_data['has_level_1'])
                    has_level_2_f = float(runner_data['has_level_2'])
                    has_level_3_f = float(runner_data['has_level_3'])

                    # === ENGINEERED DEPTH FEATURES (3 features) ===
                    # Total liquidity at each side
                    total_back_liquidity = (runner_data['back_vol_1'] +
                                           runner_data['back_vol_2'] +
                                           runner_data['back_vol_3'])
                    total_lay_liquidity = (runner_data['lay_vol_1'] +
                                          runner_data['lay_vol_2'] +
                                          runner_data['lay_vol_3'])

                    total_back_liq_norm = safe_log_norm(total_back_liquidity)
                    total_lay_liq_norm = safe_log_norm(total_lay_liquidity)

                    # Depth concentration (how much is at level 1)
                    if total_back_liquidity > 0:
                        depth_concentration = runner_data['back_vol_1'] / total_back_liquidity
                    else:
                        depth_concentration = 1.0

                    # Volume acceleration (recent vs total)
                    if runner_data['traded_vol_total'] > 0:
                        vol_acceleration = runner_data['traded_vol_60s'] / runner_data['traded_vol_total']
                    else:
                        vol_acceleration = 0.0
                    vol_accel_norm = safe_normalize(vol_acceleration, 0.0, 1.0)

                    # === POSITION TRACKING (2 features) ===
                    net_position = self._get_net_position_stake(runner_idx) / MAX_CAPITAL
                    position_pnl = self._get_position_pnl(runner_idx, runner_data['microprice']) / MAX_CAPITAL

                    # === CUSTOM VOLATILITY (1 feature - kept from V42) ===
                    if len(self.price_history) >= 2:
                        recent_prices = [p.get(runner_idx, runner_data['microprice'])
                                       for p in list(self.price_history)[-5:]]
                        price_volatility = np.std(recent_prices) if len(recent_prices) > 1 else 0.0
                    else:
                        price_volatility = 0.0
                    price_vol_norm = safe_log_norm(price_volatility)

                    # === TOTAL: 31 FEATURES PER RUNNER ===
                    obs.extend([
                        # Level 1 (4)
                        back_1, lay_1, back_vol_1, lay_vol_1,
                        # Level 2 (4)
                        back_2, lay_2, back_vol_2, lay_vol_2,
                        # Level 3 (4)
                        back_3, lay_3, back_vol_3, lay_vol_3,
                        # Pre-calculated (6)
                        microprice, ob_imbalance, rel_spread, prob_implied,
                        ret_std_5s, ret_std_20s,
                        # Trading activity (4)
                        ltp, traded_vol_total, traded_vol_60s, secs_since_last_trade,
                        # Data quality (3)
                        has_level_1_f, has_level_2_f, has_level_3_f,
                        # Engineered depth (3)
                        total_back_liq_norm, total_lay_liq_norm, depth_concentration,
                        # Volume (1)
                        vol_accel_norm,
                        # Position (2)
                        net_position, position_pnl,
                        # Custom volatility (1)
                        price_vol_norm,
                    ])
                else:
                    # No data for this runner
                    obs.extend([0.0] * 31)
            else:
                # Runner index beyond runner_count
                obs.extend([0.0] * 31)

        # GLOBAL FEATURES (11 features)
        current_exposure = self._get_total_exposure_with_liability()
        available_capital = max(0.0, self.balance - current_exposure)
        portfolio_mtm = self._get_total_unrealized_pnl()

        obs.extend([
            self.balance / MAX_CAPITAL,
            current_exposure / MAX_CAPITAL,
            available_capital / MAX_CAPITAL,
            len(self.positions) / 24.0,
            safe_normalize(self.step_idx, 0, len(self.current_race_df)),
            self.total_commission_paid / MAX_CAPITAL,
            len(self.trades_last_10_steps) / 10.0,
            safe_normalize(self.runner_count, 2, 24),
            self._get_total_net_exposure() / MAX_CAPITAL,
            self._get_position_concentration(),
            portfolio_mtm / MAX_CAPITAL
        ])

        # Verify dimension
        assert len(obs) == 755, f"Expected 755 dims, got {len(obs)}"

        return np.array(obs, dtype=np.float32)

    def step(self, action):
        """
        V43: LIABILITY-BASED with:
        - Depth checking (prevent trade explosions)
        - Volatility-based sizing
        - Stale market filtering
        - Green-up at in-play
        """

        # Get curriculum settings
        if self.curriculum_tracker is not None:
            current_min_liability = self.curriculum_tracker.get_current_min_liability()
            current_threshold = self.curriculum_tracker.get_current_action_threshold()
            self.curriculum_tracker.step()
        else:
            current_min_liability = PRODUCTION_MIN_LIABILITY
            current_threshold = PRODUCTION_ACTION_THRESHOLD

        if self.step_idx >= len(self.current_race_df):
            return self._get_observation(), 0.0, True, False, {}

        # Decode action
        runner_signals = action[:24]
        allocation_raw = action[24]
        allocation_pct = (allocation_raw + 1.0) / 2.0

        # Capital reserve
        current_exposure = self._get_total_exposure_with_liability()
        unreserved_capital = max(0.0, self.balance - current_exposure)
        available_capital = unreserved_capital * (1.0 - RESERVE_RATIO)
        trade_budget = available_capital * allocation_pct * MAX_EXPOSURE_MULTIPLIER

        # Execute trades with V43 safety checks
        trades_executed = 0
        current_row = self.current_race_df.iloc[self.step_idx]

        # Store MTM before trades
        mtm_before = self._get_total_unrealized_pnl()

        for runner_idx in range(min(24, self.runner_count)):
            signal = runner_signals[runner_idx]

            # Signal threshold
            if abs(signal) < current_threshold:
                continue

            runner_data = get_runner_data(current_row, runner_idx)
            if runner_data is None:
                continue

            # === V43 SAFETY CHECK #1: DATA QUALITY ===
            if not runner_data['has_level_1']:
                # No level 1 data - skip this runner
                continue

            # === V43 SAFETY CHECK #2: STALE MARKET ===
            if runner_data['secs_since_last_trade'] > STALE_MARKET_THRESHOLD:
                # Market too quiet - prices may be stale
                self.stale_market_violations += 1
                continue

            # === V43 SAFETY CHECK #3: HIGH VOLATILITY ===
            if runner_data['ret_std_5s'] > HIGH_VOLATILITY_THRESHOLD:
                # Volatile period - reduce position size
                volatility_multiplier = 0.3
                self.volatility_violations += 1
            else:
                volatility_multiplier = 1.0

            side = 'BACK' if signal > 0 else 'LAY'
            price = runner_data['back_1'] if side == 'BACK' else runner_data['lay_1']

            if price < 1.01 or price > 1000:
                continue

            signal_strength = abs(signal)
            runner_budget = trade_budget * signal_strength * volatility_multiplier

            if runner_budget < current_min_liability:
                continue

            # === V43 SAFETY CHECK #4: DEPTH CHECKING ===
            # Calculate available liquidity at all 3 levels
            if side == 'BACK':
                total_available = (runner_data['back_vol_1'] +
                                  runner_data['back_vol_2'] +
                                  runner_data['back_vol_3'])
            else:
                total_available = (runner_data['lay_vol_1'] +
                                  runner_data['lay_vol_2'] +
                                  runner_data['lay_vol_3'])

            # Convert budget to stake equivalent
            if side == 'BACK':
                intended_stake = runner_budget
            else:
                intended_stake = runner_budget / (price - 1.0) if price > 1.01 else 0.0

            # Don't trade if we'd consume too much of the available depth
            if intended_stake > total_available * MIN_DEPTH_RATIO:
                # Would consume >50% of depth - skip to prevent price impact
                self.depth_violations += 1
                continue

            # Calculate actual liability (capped by depth and runner budget)
            max_safe_stake = total_available * MIN_DEPTH_RATIO
            liability = min(runner_budget, available_capital * 0.3)

            # Further cap by depth
            if side == 'BACK':
                liability = min(liability, max_safe_stake)
            else:
                max_safe_liability = max_safe_stake * (price - 1.0)
                liability = min(liability, max_safe_liability)

            if liability < current_min_liability:
                continue

            success = self._execute_trade(runner_idx, side, liability, price)
            if success:
                trades_executed += 1

        # Store prices for volatility calculation
        prices = {}
        for runner_idx in range(self.runner_count):
            rd = get_runner_data(current_row, runner_idx)
            if rd:
                prices[runner_idx] = rd['microprice']
        self.price_history.append(prices)

        # Calculate rewards (same as V42)
        mtm_after = self._get_total_unrealized_pnl()
        mtm_change = mtm_after - mtm_before
        mtm_reward = (mtm_change / MAX_CAPITAL) * MTM_REWARD_SCALE

        self.pnl_history.append(mtm_change)
        sharpe_reward = self._calculate_sharpe_reward()

        self.total_mtm_reward += mtm_reward
        self.total_sharpe_reward += sharpe_reward

        capital_bonus = CAPITAL_PRESERVATION_BONUS if self.balance > (MAX_CAPITAL * 0.8) else 0.0
        activity_reward = STEP_REWARD_TRADE if trades_executed > 0 else STEP_REWARD_NO_TRADE

        step_reward = mtm_reward + sharpe_reward + capital_bonus + activity_reward

        self.previous_mtm_pnl = mtm_after
        self.peak_balance = max(self.peak_balance, self.balance)

        self.step_idx += 1

        # Check for in-play transition (green-up point)
        reached_in_play = self._check_in_play_transition()

        # Episode ends if: (1) in-play transition, or (2) data exhausted
        done = reached_in_play or (self.step_idx >= len(self.current_race_df))

        # Terminal reward
        terminal_reward = 0.0
        if done:
            # Green up all positions
            green_up_pnl = self._calculate_green_up_pnl()
            self.balance += green_up_pnl
            self.positions = {}
            terminal_reward = green_up_pnl / MAX_CAPITAL

        total_reward = step_reward + terminal_reward
        obs = self._get_observation()

        # Calculate max drawdown
        max_drawdown = ((self.peak_balance - self.balance) / self.peak_balance) * 100 if self.peak_balance > 0 else 0.0

        # Info dict
        info = {
            'trades_executed': trades_executed,
            'final_balance': self.balance,
            'final_pnl': self.balance - self.initial_balance,
            'num_trades': len(self.trades_this_episode),
            'mtm_reward': mtm_reward,
            'sharpe_reward': sharpe_reward,
            'total_mtm_reward': self.total_mtm_reward,
            'total_sharpe_reward': self.total_sharpe_reward,
            'max_drawdown': max_drawdown,
            'depth_violations': self.depth_violations,
            'volatility_violations': self.volatility_violations,
            'stale_market_violations': self.stale_market_violations,
        }

        # Episode info for callbacks
        if done:
            info['episode'] = {
                'r': total_reward,
                'l': self.step_idx,
                'realized_pnl': self.balance - self.initial_balance,
                'num_trades': len(self.trades_this_episode),
                'final_balance': self.balance,
                'mtm_reward': self.total_mtm_reward,
                'sharpe_reward': self.total_sharpe_reward,
                'max_drawdown': max_drawdown,
                'depth_violations': self.depth_violations,
                'volatility_violations': self.volatility_violations,
                'stale_market_violations': self.stale_market_violations,
            }

        return obs, total_reward, done, False, info

    def _calculate_sharpe_reward(self):
        """Sharpe ratio component."""
        if len(self.pnl_history) < 10:
            return 0.0

        recent_pnl = list(self.pnl_history)[-20:]
        mean_pnl = np.mean(recent_pnl)
        std_pnl = np.std(recent_pnl)

        if std_pnl < 1e-8:
            return 0.0

        sharpe = mean_pnl / std_pnl
        sharpe_clipped = np.clip(sharpe, -3.0, 3.0)

        return sharpe_clipped * SHARPE_REWARD_SCALE

    def _get_total_unrealized_pnl(self):
        """Mark-to-market P&L."""
        if self.step_idx >= len(self.current_race_df):
            return 0.0

        current_row = self.current_race_df.iloc[self.step_idx]
        total_mtm = 0.0

        for runner_idx, pos in self.positions.items():
            if abs(pos['net_stake']) < 1e-6:
                continue

            runner_data = get_runner_data(current_row, runner_idx)
            if runner_data is None:
                continue

            current_price = runner_data['microprice']
            mtm_pnl = self._get_position_pnl(runner_idx, current_price)
            total_mtm += mtm_pnl

        return total_mtm

    def _execute_trade(self, runner_idx, side, liability, price):
        """Execute trade with position netting."""

        if side == 'BACK':
            stake = liability
        else:
            stake = liability / (price - 1.0) if price > 1.01 else 0.0

        if stake < 0.01:
            return False

        # Position netting
        if runner_idx not in self.positions:
            self.positions[runner_idx] = {
                'net_stake': 0.0,
                'weighted_price': 0.0,
                'total_back_stake': 0.0,
                'total_lay_liability': 0.0
            }

        pos = self.positions[runner_idx]

        if side == 'BACK':
            new_back = pos['total_back_stake'] + stake
            if new_back > 0:
                pos['weighted_price'] = (
                    (pos['weighted_price'] * pos['total_back_stake'] + price * stake) / new_back
                )
            pos['total_back_stake'] = new_back
            pos['net_stake'] += stake
        else:
            pos['total_lay_liability'] += liability
            pos['net_stake'] -= stake
            if pos['weighted_price'] == 0:
                pos['weighted_price'] = price

        # Logging
        trade_info = {
            'step': self.step_idx,
            'runner': runner_idx,
            'side': side,
            'price': price,
            'stake': stake,
            'liability': liability
        }
        self.trades_this_episode.append(trade_info)
        self.trades_last_10_steps.append(trade_info)

        return True

    def _check_in_play_transition(self):
        """Check if we've reached the in-play transition point."""
        if self.step_idx >= len(self.current_race_df):
            return False

        current_row = self.current_race_df.iloc[self.step_idx]

        # Check seconds_to_start if available
        seconds_to_start = current_row.get('seconds_to_start', None)
        if seconds_to_start is not None:
            return seconds_to_start <= 0

        # Fallback: Use time-based threshold (last 5% of data)
        total_steps = len(self.current_race_df)
        in_play_threshold = int(total_steps * 0.95)
        return self.step_idx >= in_play_threshold

    def _calculate_green_up_pnl(self):
        """Calculate P&L from greening up all NET positions at current prices."""
        if not self.positions:
            return 0.0

        total_green_up_pnl = 0.0
        current_row = self.current_race_df.iloc[self.step_idx]

        for runner_idx, position in self.positions.items():
            runner_data = get_runner_data(current_row, runner_idx)

            if runner_data is None:
                continue

            net_stake = position.get('net_stake', 0.0)
            weighted_price = position.get('weighted_price', 0.0)

            if abs(net_stake) < 0.01 or weighted_price < 1.01:
                continue

            current_price = runner_data['microprice']

            if current_price < 1.01:
                continue

            # Calculate green-up P&L
            if net_stake > 0:
                # NET BACK position
                pnl = net_stake * (weighted_price - current_price) / current_price
            else:
                # NET LAY position
                liability = abs(net_stake) * (weighted_price - 1)
                pnl = liability * (1.0 / weighted_price - 1.0 / current_price)

            # Apply commission on profitable green-ups only
            if pnl > 0:
                commission = pnl * COMMISSION_RATE
                pnl_after_commission = pnl - commission
            else:
                pnl_after_commission = pnl

            total_green_up_pnl += pnl_after_commission

        return total_green_up_pnl

    def _get_net_position_stake(self, runner_id):
        if runner_id not in self.positions:
            return 0.0
        return self.positions[runner_id].get('net_stake', 0.0)

    def _get_position_pnl(self, runner_id, current_price):
        if runner_id not in self.positions:
            return 0.0

        pos = self.positions[runner_id]
        net_stake = pos['net_stake']

        if abs(net_stake) < 1e-6:
            return 0.0

        entry_price = pos['weighted_price']

        if net_stake > 0:
            pnl = net_stake * (current_price - entry_price)
        else:
            abs_stake = abs(net_stake)
            pnl = abs_stake * (entry_price - current_price)

        return pnl

    def _get_total_exposure_with_liability(self):
        total_exposure = 0.0

        for runner_idx, pos in self.positions.items():
            net_stake = pos['net_stake']
            entry_price = pos['weighted_price']

            if net_stake > 0:
                total_exposure += abs(net_stake)
            else:
                abs_stake = abs(net_stake)
                liability = abs_stake * (entry_price - 1.0) if entry_price > 1.0 else abs_stake
                total_exposure += liability

        return total_exposure

    def _get_total_net_exposure(self):
        return sum(pos['net_stake'] for pos in self.positions.values())

    def _get_position_concentration(self):
        if len(self.positions) == 0:
            return 0.0

        exposures = []
        for pos in self.positions.values():
            net_stake = pos['net_stake']
            if abs(net_stake) > 1e-6:
                exposures.append(abs(net_stake))

        if len(exposures) == 0:
            return 0.0

        total = sum(exposures)
        if total < 1e-6:
            return 0.0

        proportions = [e / total for e in exposures]
        herfindahl = sum(p**2 for p in proportions)

        return herfindahl

# ============================================================
# NO-TRADE STREAK WRAPPER
# ============================================================

class NoTradeStreakWrapper(gym.Wrapper):
    """Penalize consecutive no-trade episodes."""

    def __init__(self, env):
        super().__init__(env)
        self.consecutive_no_trade_episodes = 0
        print(f"[WRAPPER] NoTradeStreakWrapper initialized")

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

    def step(self, action):
        obs, reward, done, truncated, info = self.env.step(action)

        if done:
            base_env = self.env
            while hasattr(base_env, 'env'):
                base_env = base_env.env

            num_trades = len(base_env.trades_this_episode)
            had_trades = num_trades > 0

            if had_trades:
                self.consecutive_no_trade_episodes = 0
            else:
                self.consecutive_no_trade_episodes += 1

            if self.consecutive_no_trade_episodes > 0:
                exponent = min(self.consecutive_no_trade_episodes - 1, 5)
                penalty = -1.0 * (2 ** exponent)
            else:
                penalty = 0.0

            reward = float(reward) + penalty

            if 'episode' not in info:
                info['episode'] = {}

            info['episode']['no_trade_streak'] = self.consecutive_no_trade_episodes
            info['episode']['no_trade_penalty'] = penalty
            info['episode']['had_trades'] = had_trades
            info['episode']['num_trades'] = num_trades
            info['episode']['final_balance'] = info.get('final_balance', 1000.0)
            info['episode']['final_pnl'] = info.get('final_pnl', 0.0)

            info['no_trade_streak'] = self.consecutive_no_trade_episodes
            info['no_trade_penalty'] = penalty
            info['had_trades'] = had_trades
            info['num_trades'] = num_trades
            info['final_balance'] = info.get('final_balance', 1000.0)
            info['final_pnl'] = info.get('final_pnl', 0.0)

        return obs, reward, done, truncated, info

# ============================================================
# DATA LOADING
# ============================================================

def load_race_files(data_dir):
    """Load all parquet files with validation."""
    race_files = []
    skipped_files = 0

    print(f"\n[DATA] Loading parquet files from: {data_dir}")

    for file in os.listdir(data_dir):
        if file.endswith('.parquet'):
            filepath = os.path.join(data_dir, file)

            try:
                df = pd.read_parquet(filepath)

                if len(df) < 10:
                    skipped_files += 1
                    continue

                # Validation: Check for run[0].back_price_1 format
                has_run0 = 'run[0].back_price_1' in df.columns

                if not has_run0:
                    if skipped_files < 3:
                        print(f"⚠️  Skipping {file}: No run[0] columns found")
                    skipped_files += 1
                    continue

                race_files.append(filepath)

            except Exception as e:
                if skipped_files < 3:
                    print(f"⚠️  Error loading {file}: {str(e)[:50]}")
                skipped_files += 1
                continue

    print(f"[DATA] Loaded {len(race_files)} valid race files")
    if skipped_files > 0:
        print(f"[DATA] Skipped {skipped_files} invalid files")

    if len(race_files) == 0:
        raise RuntimeError("No valid race files found!")

    return race_files

# ============================================================
# TRAINING METRICS CALLBACK (V43)
# ============================================================

class TrainingMetricsCallback(BaseCallback):
    """V43: Track performance + violations."""

    def __init__(self, log_interval=1, save_path=None, curriculum_tracker=None):
        super().__init__()
        self.log_interval = log_interval
        self.save_path = save_path
        self.curriculum_tracker = curriculum_tracker
        self.episode_count = 0
        self.metrics = []

    def _on_step(self) -> bool:
        if self.num_timesteps % 500 == 0:
            print(f"🔍 Training callback alive at step {self.num_timesteps}, episodes: {self.episode_count}")

        dones = self.locals.get('dones', None)

        if dones is None:
            return True

        if isinstance(dones, np.ndarray):
            episode_done = bool(dones[0])
        elif isinstance(dones, list):
            episode_done = bool(dones[0])
        else:
            episode_done = bool(dones)

        if episode_done:
            print(f"🎯 Episode {self.episode_count + 1} completed at step {self.num_timesteps}")
            self.episode_count += 1

            infos = self.locals.get('infos', [{}])
            info = infos[0] if len(infos) > 0 else {}

            env = self.training_env
            while hasattr(env, 'env'):
                env = env.env
            if hasattr(env, 'envs'):
                env = env.envs[0]
                while hasattr(env, 'env'):
                    env = env.env

            # Read from info['episode'] first
            if 'episode' in info and isinstance(info['episode'], dict):
                episode_info = info['episode']
                streak = episode_info.get('no_trade_streak', 0)
                penalty = episode_info.get('no_trade_penalty', 0.0)
                had_trades = episode_info.get('had_trades', False)
                num_trades = episode_info.get('num_trades', 0)
                final_balance = episode_info.get('final_balance', 1000.0)
                final_pnl = episode_info.get('final_pnl', 0.0)
                depth_violations = episode_info.get('depth_violations', 0)
                volatility_violations = episode_info.get('volatility_violations', 0)
                stale_market_violations = episode_info.get('stale_market_violations', 0)
            else:
                streak = info.get('no_trade_streak', 0)
                penalty = info.get('no_trade_penalty', 0.0)
                had_trades = info.get('had_trades', False)
                num_trades = info.get('num_trades', 0)
                final_balance = info.get('final_balance', 1000.0)
                final_pnl = info.get('final_pnl', 0.0)
                depth_violations = info.get('depth_violations', 0)
                volatility_violations = info.get('volatility_violations', 0)
                stale_market_violations = info.get('stale_market_violations', 0)

            max_drawdown = (env.peak_balance - final_balance) / env.peak_balance if env.peak_balance > 0 else 0.0

            if self.episode_count % self.log_interval == 0:
                metrics = {
                    'Episode': self.episode_count,
                    'Step': self.num_timesteps,
                    'Balance': final_balance,
                    'Num_Trades': num_trades,
                    'Realized_PnL': final_pnl,
                    'No_Trade_Streak': streak,
                    'No_Trade_Penalty': penalty,
                    'Had_Trades': had_trades,
                    'Max_Drawdown': max_drawdown,
                    'MTM_Reward': info.get('total_mtm_reward', 0.0),
                    'Sharpe_Reward': info.get('total_sharpe_reward', 0.0),
                    'Depth_Violations': depth_violations,
                    'Volatility_Violations': volatility_violations,
                    'Stale_Market_Violations': stale_market_violations,
                }

                self.metrics.append(metrics)
                print(f"📝 Metrics collected: Episode {self.episode_count}")

                if self.episode_count % 10 == 0:
                    recent = self.metrics[-10:]
                    trade_rate = sum(1 for m in recent if m['Num_Trades'] > 0) / len(recent) * 100
                    avg_pnl = np.mean([m['Realized_PnL'] for m in recent])
                    avg_drawdown = np.mean([m['Max_Drawdown'] for m in recent])
                    avg_depth_viol = np.mean([m['Depth_Violations'] for m in recent])
                    avg_vol_viol = np.mean([m['Volatility_Violations'] for m in recent])

                    print(f"\n📊 Ep {self.episode_count} | Step {self.num_timesteps:,}")

                    if self.curriculum_tracker is not None:
                        print(f"   {self.curriculum_tracker.get_status_string()}")

                    print(f"   Trade Rate: {trade_rate:.0f}%")
                    print(f"   Avg P&L: ${avg_pnl:.2f}")
                    print(f"   Avg Drawdown: {avg_drawdown*100:.1f}%")
                    print(f"   Avg Depth Violations: {avg_depth_viol:.1f}")
                    print(f"   Avg Volatility Violations: {avg_vol_viol:.1f}")

                    if trade_rate > 0:
                        traded = [m for m in recent if m['Num_Trades'] > 0]
                        avg_trades = np.mean([m['Num_Trades'] for m in traded])
                        print(f"   Avg Trades/Race: {avg_trades:.1f}")

                if self.save_path:
                    try:
                        pd.DataFrame(self.metrics).to_csv(self.save_path, index=False)
                        if self.episode_count % 10 == 0:
                            print(f"💾 Saved {len(self.metrics)} episodes to CSV")
                    except Exception as e:
                        print(f"❌ Save failed: {str(e)}")

        return True

# ============================================================
# VALIDATION CALLBACK (V43 - FIXED)
# ============================================================

class ValidationCallback(BaseCallback):
    """V43: Validation with proper unwrapping + violation tracking."""

    def __init__(self, val_env, val_interval=25000, save_path=None):
        super().__init__()
        self.val_env = val_env
        self.val_interval = val_interval
        self.save_path = save_path
        self.val_metrics = []

        # Create CSV file immediately
        if self.save_path:
            pd.DataFrame(columns=[
                'Step', 'Val_Episode', 'Num_Trades', 'Realized_PnL',
                'Final_Balance', 'MTM_Reward', 'Sharpe_Reward',
                'Depth_Violations', 'Volatility_Violations', 'Stale_Market_Violations'
            ]).to_csv(self.save_path, index=False)
            print(f"✅ Validation CSV created: {self.save_path}")

    def _on_step(self) -> bool:
        """Run validation with proper environment unwrapping."""
        if self.num_timesteps % self.val_interval == 0 and self.num_timesteps > 0:
            print(f"\n{'='*60}")
            print(f"🔍 VALIDATION @ {self.num_timesteps:,} steps")
            print(f"{'='*60}")

            val_results = []

            for ep in range(10):
                try:
                    obs, _ = self.val_env.reset()
                    done = False

                    # Unwrap to get actual environment
                    actual_env = self.val_env
                    while hasattr(actual_env, 'env'):
                        actual_env = actual_env.env

                    # Run episode
                    steps = 0
                    while not done and steps < 2000:
                        action, _ = self.model.predict(obs, deterministic=True)
                        obs, reward, done, truncated, info = self.val_env.step(action)
                        steps += 1
                        done = done or truncated

                    # Try to get from info['episode'] first
                    if 'episode' in info:
                        ep_info = info['episode']
                        result = {
                            'Step': self.num_timesteps,
                            'Val_Episode': ep + 1,
                            'Num_Trades': ep_info.get('num_trades', len(actual_env.trades_this_episode)),
                            'Realized_PnL': ep_info.get('realized_pnl', actual_env.balance - actual_env.initial_balance),
                            'Final_Balance': ep_info.get('final_balance', actual_env.balance),
                            'MTM_Reward': ep_info.get('mtm_reward', 0.0),
                            'Sharpe_Reward': ep_info.get('sharpe_reward', 0.0),
                            'Depth_Violations': ep_info.get('depth_violations', 0),
                            'Volatility_Violations': ep_info.get('volatility_violations', 0),
                            'Stale_Market_Violations': ep_info.get('stale_market_violations', 0),
                        }
                    else:
                        # Fallback to unwrapped environment
                        result = {
                            'Step': self.num_timesteps,
                            'Val_Episode': ep + 1,
                            'Num_Trades': len(actual_env.trades_this_episode),
                            'Realized_PnL': actual_env.balance - actual_env.initial_balance,
                            'Final_Balance': actual_env.balance,
                            'MTM_Reward': getattr(actual_env, 'total_mtm_reward', 0.0),
                            'Sharpe_Reward': getattr(actual_env, 'total_sharpe_reward', 0.0),
                            'Depth_Violations': getattr(actual_env, 'depth_violations', 0),
                            'Volatility_Violations': getattr(actual_env, 'volatility_violations', 0),
                            'Stale_Market_Violations': getattr(actual_env, 'stale_market_violations', 0),
                        }

                    val_results.append(result)

                except Exception as e:
                    print(f"⚠️  Val episode {ep+1} failed: {str(e)[:60]}")
                    continue

            # Save and report results
            if val_results:
                df = pd.DataFrame(val_results)

                # Append to CSV
                if self.save_path:
                    df.to_csv(self.save_path, mode='a', header=False, index=False)

                # Print summary
                trade_rate = (df['Num_Trades'] > 0).sum() / len(df) * 100
                mean_pnl = df['Realized_PnL'].mean()
                mean_trades = df['Num_Trades'].mean()
                win_rate = (df['Realized_PnL'] > 0).sum() / len(df) * 100
                mean_depth_viol = df['Depth_Violations'].mean()

                print(f"\n📊 Validation Summary ({len(val_results)} episodes):")
                print(f"   Mean P&L: ${mean_pnl:.2f}")
                print(f"   Mean Trades: {mean_trades:.1f}")
                print(f"   Win Rate: {win_rate:.1f}%")
                print(f"   Trade Rate: {trade_rate:.1f}%")
                print(f"   Avg Depth Violations: {mean_depth_viol:.1f}")
                print(f"💾 Saved to {self.save_path}")
            else:
                print(f"\n❌ No validation results collected")

            print("="*60 + "\n")

        return True


class CheckpointCallback(BaseCallback):
    """Save model checkpoints at regular intervals."""

    def __init__(self, save_freq, save_path):
        super().__init__()
        self.save_freq = save_freq
        self.save_path = save_path

    def _on_step(self) -> bool:
        if self.num_timesteps % self.save_freq == 0 and self.num_timesteps > 0:
            model_path = f"{self.save_path}/model_{self.num_timesteps}"
            self.model.save(model_path)
            print(f"💾 Model saved: {model_path}")
        return True

print("\n✅ V43 Environment and callbacks loaded!")
print("✅ 755-dimensional observation space")
print("✅ All 26 features per runner")
print("✅ Depth checking, volatility filtering, stale market detection")
print("✅ Green-up strategy at in-play transition")

Model Version: V43_Full_Features_Green_Up
Algorithm: SAC + ALL Features + Green-up

✨ Features:
   • ALL 26 available features (100% utilization)
   • 755-dimensional observation space
   • Order book depth (levels 1-3)
   • Pre-calculated volatility (ret_std_5s, ret_std_20s)
   • Pre-calculated market metrics
   • Trading activity signals
   • Data quality indicators
   • Green-up at in-play transition
   • Depth-based trade filtering
   • Volatility-based position sizing
   • Stale market detection

✅ V43 Environment and callbacks loaded!
✅ 755-dimensional observation space
✅ All 26 features per runner
✅ Depth checking, volatility filtering, stale market detection
✅ Green-up strategy at in-play transition


In [ ]:
### CELL 5 - SETUP TRAINING (V43 - FULL FEATURES) ###

# V43: All features + green-up strategy
# 755-dimensional observations

# Load race files
print("\n📂 Loading race files...")
train_files = load_race_files(DATA_DIR)
val_files = train_files[-100:]  # Last 100 for validation
train_files = train_files[:-100]  # Rest for training

print(f"Training files: {len(train_files)}")
print(f"Validation files: {len(val_files)}")

# Create curriculum tracker
print("\n📚 Creating curriculum tracker...")
curriculum = CurriculumTracker(
    total_steps=CURRICULUM_TOTAL_STEPS,
    warmup_steps=CURRICULUM_WARMUP_STEPS
)
print(f"✅ Curriculum: {CURRICULUM_WARMUP_STEPS:,} warmup → {CURRICULUM_TOTAL_STEPS:,} total")

# Create BARE environments (NO DummyVecEnv)
print("\n🔧 Creating environments...")

# Training environment
train_env = MarketMakingEnv(train_files, curriculum_tracker=curriculum)
train_env = Monitor(train_env)
train_env = NoTradeStreakWrapper(train_env)

print("✅ Training environment created")
print("   Wrapper order: NoTradeStreakWrapper → Monitor → MarketMakingEnv")
print("   Observation dims: 755")

# Validation environment (NO VecNormalize!)
val_env = MarketMakingEnv(val_files, curriculum_tracker=None)
val_env = Monitor(val_env)

print("✅ Validation environment created")
print("   Observation dims: 755")

# Create callbacks
print("\n📊 Setting up callbacks...")

training_callback = TrainingMetricsCallback(
    log_interval=1,
    save_path=f"{BASE_PATH}/training_metrics.csv",
    curriculum_tracker=curriculum
)

validation_callback = ValidationCallback(
    val_env=val_env,
    val_interval=25000,
    save_path=f"{BASE_PATH}/validation_metrics.csv"
)

checkpoint_callback = CheckpointCallback(
    save_freq=50000,
    save_path=BASE_PATH
)

callbacks = CallbackList([
    training_callback,
    validation_callback,
    checkpoint_callback
])

print("✅ Callbacks configured")
print(f"   Training metrics: {BASE_PATH}/training_metrics.csv")
print(f"   Validation metrics: {BASE_PATH}/validation_metrics.csv")

# Create SAC model
print("\n🤖 Creating SAC model...")

model = SAC(
    "MlpPolicy",
    train_env,
    learning_rate=SAC_LEARNING_RATE,
    buffer_size=SAC_BUFFER_SIZE,
    learning_starts=SAC_LEARNING_STARTS,
    batch_size=SAC_BATCH_SIZE,
    tau=SAC_TAU,
    gamma=SAC_GAMMA,
    train_freq=SAC_TRAIN_FREQ,
    gradient_steps=SAC_GRADIENT_STEPS,
    ent_coef=SAC_ENT_COEF,
    verbose=1,
    tensorboard_log=f"{BASE_PATH}/logs"
)

print("✅ SAC model created")
print(f"   Policy: MlpPolicy")
print(f"   Observation dims: 755 (vs V42's 323)")
print(f"   Learning rate: {SAC_LEARNING_RATE}")
print(f"   Buffer size: {SAC_BUFFER_SIZE:,}")
print(f"   Entropy coef: {SAC_ENT_COEF}")

print("\n" + "="*60)
print("✅ V43 Setup complete - ready for training!")
print("="*60)
print("\n🎯 V43 Improvements over V42:")
print("   • 755 dims (vs 323): +133% more information")
print("   • Order book depth (levels 1-3): Prevent trade explosions")
print("   • Pre-calculated volatility: Better risk management")
print("   • Pre-calculated metrics: Better signals")
print("   • Depth checking: Block trades without liquidity")
print("   • Volatility filtering: Reduce position in chaos")
print("   • Stale market detection: Avoid outdated prices")
print("\n💡 Expected improvements:")
print("   • Win rate: 18.5% → 45-60%")
print("   • Trade explosions: 13.3% → <3%")
print("   • Mean P&L: -$104 → +$30-100")
print("="*60)


📂 Loading race files...

[DATA] Loading parquet files from: /content/drive/MyDrive/race_out
⚠️  Error loading 2025-12-23_Ashburton_Race3_1.251967693.parquet: Could not open Parquet input source '<Buffer>': Co
⚠️  Error loading 2025-12-24_Ashburton_Race4_1.251967695.parquet: Corrupt snappy compressed data.
[DATA] Loaded 3205 valid race files
[DATA] Skipped 13 invalid files
Training files: 3105
Validation files: 100

📚 Creating curriculum tracker...
✅ Curriculum: 20,000 warmup → 200,000 total

🔧 Creating environments...

[ENV INIT] V43 with ALL features (755 dimensions)
[ENV] Curriculum tracker attached ✅
[WRAPPER] NoTradeStreakWrapper initialized
✅ Training environment created
   Wrapper order: NoTradeStreakWrapper → Monitor → MarketMakingEnv
   Observation dims: 755

[ENV INIT] V43 with ALL features (755 dimensions)
[ENV] No curriculum (fixed constraints)
✅ Validation environment created
   Observation dims: 755

📊 Setting up callbacks...
✅ Validation CSV created: /content/drive/MyDri

In [ ]:
# # ============================================================================
# # CELL 5 - HELPER FUNCTIONS (FIXED FOR ACTUAL DATA STRUCTURE)
# # ============================================================================
# #
# # DROP-IN REPLACEMENT for Cell 5
# #
# # FIXES APPLIED:
# # - Maps to ACTUAL column names in your data
# # - back_size_1/2/3 instead of back_vol_1/2/3
# # - lay_size_1/2/3 instead of lay_vol_1/2/3
# # - last_traded_price instead of ltp
# # - Uses pre-calculated microprice, ob_imbalance, rel_spread
# #
# # ============================================================================

# import numpy as np
# import warnings
# warnings.filterwarnings('ignore')

# print("="*80)
# print("🔧 LOADING HELPER FUNCTIONS")
# print("="*80)

# # Type-safe helpers
# def to_float(val, default=0.0):
#     """Convert to float safely."""
#     try:
#         if val is None or (isinstance(val, float) and np.isnan(val)):
#             return default
#         return float(val)
#     except (ValueError, TypeError):
#         return default

# def safe_normalize(val, min_val, max_val):
#     """Normalize value to [0, 1] range."""
#     if val is None or val == 0.0:
#         return 0.0
#     val = max(min_val, min(max_val, val))
#     return (val - min_val) / (max_val - min_val + 1e-8)

# def safe_log_norm(val, default=0.0):
#     """Log-normalize value."""
#     if val is None or val <= 0:
#         return default
#     return np.log1p(val) / 10.0

# # V41 column extraction (FIXED for actual data structure)
# def get_runner_data(row, runner_idx):
#     """Extract runner data with run[idx] format.

#     UPDATED to use actual column names:
#     - back_size instead of back_vol
#     - lay_size instead of lay_vol
#     - last_traded_price instead of ltp
#     - traded_vol_total instead of ltv
#     - ob_imbalance instead of calculating wom
#     - rel_spread (already calculated)
#     """
#     try:
#         prefix = f'run[{runner_idx}].'

#         # Check if runner exists
#         back_col = f'{prefix}back_price_1'
#         if back_col not in row.index:
#             return None

#         # Extract prices
#         back_1 = to_float(row.get(f'{prefix}back_price_1'))
#         lay_1 = to_float(row.get(f'{prefix}lay_price_1'))

#         if back_1 <= 1.0 or lay_1 <= 1.0:
#             return None

#         # Volumes/Sizes - FIXED NAMES
#         back_vol_1 = to_float(row.get(f'{prefix}back_size_1'), 0.0)
#         lay_vol_1 = to_float(row.get(f'{prefix}lay_size_1'), 0.0)
#         back_vol_2 = to_float(row.get(f'{prefix}back_size_2'), 0.0)
#         lay_vol_2 = to_float(row.get(f'{prefix}lay_size_2'), 0.0)
#         back_vol_3 = to_float(row.get(f'{prefix}back_size_3'), 0.0)
#         lay_vol_3 = to_float(row.get(f'{prefix}lay_size_3'), 0.0)

#         # Last traded - FIXED NAME
#         ltp = to_float(row.get(f'{prefix}last_traded_price'), back_1)

#         # Use pre-calculated microprice if available
#         microprice = to_float(row.get(f'{prefix}microprice'))
#         if microprice <= 0:
#             # Fallback: calculate if not available
#             total_vol = back_vol_1 + lay_vol_1
#             if total_vol > 0:
#                 microprice = (back_1 * lay_vol_1 + lay_1 * back_vol_1) / total_vol
#             else:
#                 microprice = (back_1 + lay_1) / 2.0

#         # Use pre-calculated order book imbalance (like wom)
#         wom = to_float(row.get(f'{prefix}ob_imbalance'), 0.5)
#         # If not available or invalid, calculate from volumes
#         if wom < 0 or wom > 1:
#             total_vol = back_vol_1 + lay_vol_1
#             if total_vol > 0:
#                 wom = back_vol_1 / total_vol
#             else:
#                 wom = 0.5

#         # Use pre-calculated relative spread
#         spread = to_float(row.get(f'{prefix}rel_spread'))
#         if spread <= 0:
#             # Fallback: calculate if not available
#             spread = lay_1 - back_1

#         # Total matched volume
#         total_matched = to_float(row.get(f'{prefix}traded_vol_total'), 0.0)

#         return {
#             'back_1': back_1,
#             'lay_1': lay_1,
#             'back_vol_1': back_vol_1,
#             'lay_vol_1': lay_vol_1,
#             'back_vol_2': back_vol_2,
#             'lay_vol_2': lay_vol_2,
#             'back_vol_3': back_vol_3,
#             'lay_vol_3': lay_vol_3,
#             'microprice': microprice,
#             'spread': spread,
#             'wom': wom,
#             'total_matched': total_matched,
#             'ltp': ltp,
#             'ltv': total_matched,
#         }
#     except Exception as e:
#         return None

# print("✅ Helper functions loaded")
# print("   • to_float()")
# print("   • safe_normalize()")
# print("   • safe_log_norm()")
# print("   • get_runner_data() - FIXED for actual data structure")
# print("   • Maps: back_size→back_vol, lay_size→lay_vol")
# print("   • Uses: microprice, ob_imbalance, rel_spread")

# ============================================================================
# CELL 6 - HELPER FUNCTIONS (FINAL CORRECTED VERSION)
# ============================================================================
#
# DROP-IN REPLACEMENT for Cell 6
#
# COMPLETE FIX - Includes ALL required keys:
#
# COLUMN NAME FIXES:
# - back_size_1/2/3 instead of back_vol_1/2/3
# - lay_size_1/2/3 instead of lay_vol_1/2/3
# - last_traded_price instead of ltp
# - traded_vol_total → both 'total_matched' AND 'ltv'
# - Uses pre-calculated: microprice, ob_imbalance, rel_spread
#
# CRITICAL KEYS ADDED:
# - 'ltv' → Last traded volume (for observations)
# - 'is_winner' → Race outcome (for settlement)
#
# ============================================================================

import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("🔧 LOADING HELPER FUNCTIONS")
print("="*80)

# Type-safe helpers
def to_float(val, default=0.0):
    """Convert to float safely."""
    try:
        if val is None or (isinstance(val, float) and np.isnan(val)):
            return default
        return float(val)
    except (ValueError, TypeError):
        return default

def safe_normalize(val, min_val, max_val):
    """Normalize value to [0, 1] range."""
    if val is None or val == 0.0:
        return 0.0
    val = max(min_val, min(max_val, val))
    return (val - min_val) / (max_val - min_val + 1e-8)

def safe_log_norm(val, default=0.0):
    """Log-normalize value."""
    if val is None or val <= 0:
        return default
    return np.log1p(val) / 10.0

# V41 column extraction (COMPLETE FIX)
def get_runner_data(row, runner_idx):
    """Extract runner data with run[idx] format.

    COMPLETE VERSION with all required keys:
    - Maps actual column names (back_size, lay_size, etc.)
    - Returns ALL keys needed by environment:
      * back_1, lay_1 (prices)
      * back_vol_1, lay_vol_1, vol_2, vol_3 (volumes)
      * ltp, ltv (last traded)
      * microprice, wom (derived)
      * is_winner (race outcome)
      * spread, total_matched (extra)
    """
    try:
        prefix = f'run[{runner_idx}].'

        # Check if runner exists
        back_col = f'{prefix}back_price_1'
        if back_col not in row.index:
            return None

        # Extract prices
        back_1 = to_float(row.get(f'{prefix}back_price_1'))
        lay_1 = to_float(row.get(f'{prefix}lay_price_1'))

        if back_1 <= 1.0 or lay_1 <= 1.0:
            return None

        # Volumes/Sizes - FIXED NAMES (back_size → back_vol)
        back_vol_1 = to_float(row.get(f'{prefix}back_size_1'), 0.0)
        lay_vol_1 = to_float(row.get(f'{prefix}lay_size_1'), 0.0)
        back_vol_2 = to_float(row.get(f'{prefix}back_size_2'), 0.0)
        lay_vol_2 = to_float(row.get(f'{prefix}lay_size_2'), 0.0)
        back_vol_3 = to_float(row.get(f'{prefix}back_size_3'), 0.0)
        lay_vol_3 = to_float(row.get(f'{prefix}lay_size_3'), 0.0)

        # Last traded price - FIXED NAME
        ltp = to_float(row.get(f'{prefix}last_traded_price'), back_1)

        # Use pre-calculated microprice if available
        microprice = to_float(row.get(f'{prefix}microprice'))
        if microprice <= 0:
            # Fallback: calculate if not available
            total_vol = back_vol_1 + lay_vol_1
            if total_vol > 0:
                microprice = (back_1 * lay_vol_1 + lay_1 * back_vol_1) / total_vol
            else:
                microprice = (back_1 + lay_1) / 2.0

        # Use pre-calculated order book imbalance (like wom)
        wom = to_float(row.get(f'{prefix}ob_imbalance'), 0.5)
        # If not available or invalid, calculate from volumes
        if wom < 0 or wom > 1:
            total_vol = back_vol_1 + lay_vol_1
            if total_vol > 0:
                wom = back_vol_1 / total_vol
            else:
                wom = 0.5

        # Use pre-calculated relative spread
        spread = to_float(row.get(f'{prefix}rel_spread'))
        if spread <= 0:
            # Fallback: calculate if not available
            spread = lay_1 - back_1

        # Total matched volume (used for both 'total_matched' and 'ltv')
        total_matched = to_float(row.get(f'{prefix}traded_vol_total'), 0.0)

        # Winner flag (critical for settlement at race end)
        is_winner = to_float(row.get(f'{prefix}is_winner'), 0.0)

        # Return ALL required keys
        return {
            # Prices (required)
            'back_1': back_1,
            'lay_1': lay_1,

            # Volumes - all 3 depth levels (vol_1 required, vol_2/3 optional)
            'back_vol_1': back_vol_1,
            'lay_vol_1': lay_vol_1,
            'back_vol_2': back_vol_2,
            'lay_vol_2': lay_vol_2,
            'back_vol_3': back_vol_3,
            'lay_vol_3': lay_vol_3,

            # Derived price features (required)
            'microprice': microprice,
            'wom': wom,

            # Last traded (required)
            'ltp': ltp,
            'ltv': total_matched,  # ← CRITICAL: Needed for observations

            # Settlement (required)
            'is_winner': is_winner,  # ← CRITICAL: Needed for race outcome

            # Extra features (optional but included)
            'spread': spread,
            'total_matched': total_matched,
        }
    except Exception as e:
        return None

print("✅ Helper functions loaded")
print("   • to_float()")
print("   • safe_normalize()")
print("   • safe_log_norm()")
print("   • get_runner_data() - COMPLETE FIXED VERSION")
print("")
print("   Column mappings:")
print("   • back_size → back_vol")
print("   • lay_size → lay_vol")
print("   • last_traded_price → ltp")
print("   • traded_vol_total → ltv & total_matched")
print("   • ob_imbalance → wom")
print("   • rel_spread → spread")
print("")
print("   ✅ All required keys included:")
print("   • back_1, lay_1, back_vol_1, lay_vol_1")
print("   • ltp, ltv, microprice, wom")
print("   • is_winner (for settlement)")
print("   • Plus: vol_2/3, spread, total_matched")

🔧 LOADING HELPER FUNCTIONS
✅ Helper functions loaded
   • to_float()
   • safe_normalize()
   • safe_log_norm()
   • get_runner_data() - COMPLETE FIXED VERSION

   Column mappings:
   • back_size → back_vol
   • lay_size → lay_vol
   • last_traded_price → ltp
   • traded_vol_total → ltv & total_matched
   • ob_imbalance → wom
   • rel_spread → spread

   ✅ All required keys included:
   • back_1, lay_1, back_vol_1, lay_vol_1
   • ltp, ltv, microprice, wom
   • is_winner (for settlement)
   • Plus: vol_2/3, spread, total_matched


In [ ]:
### MTM/Sharpe Fix Verification ###

import numpy as np

# Get unwrapped env
env = train_env
while hasattr(env, 'env'):
    env = env.env

# Reset and run one episode
obs, _ = train_env.reset()
print("Starting episode...")

for step in range(50):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = train_env.step(action)

    if step % 10 == 0:
        print(f"Step {step}: Total MTM={info.get('total_mtm_reward', 0):.4f}, Total Sharpe={info.get('total_sharpe_reward', 0):.4f}")

    if done:
        break

print(f"\nEpisode completed at step {step}")
print(f"Final Total MTM: {info.get('total_mtm_reward', 0):.4f}")
print(f"Final Total Sharpe: {info.get('total_sharpe_reward', 0):.4f}")

if info.get('total_mtm_reward', 0) != 0:
    print("\n✅ MTM ACCUMULATION WORKING!")
else:
    print("\n⚠️  MTM still zero (may be normal if no positions)")

if info.get('total_sharpe_reward', 0) != 0:
    print("✅ SHARPE ACCUMULATION WORKING!")
else:
    print("⚠️  Sharpe still zero (may be normal if <10 steps)")

KeyError: 'back_2'

In [ ]:
### CRITICAL TEST - What threshold is actually being used? ###

import numpy as np

print("="*80)
print("CHECKING ACTUAL THRESHOLD IN USE")
print("="*80)

# Get the actual curriculum from the environment
env = train_env
while hasattr(env, 'env'):
    env = env.env

if hasattr(env, 'curriculum_tracker'):
    print(f"\nCurriculum tracker values:")
    print(f"  threshold_start: {env.curriculum_tracker.threshold_start:.3f}")
    print(f"  threshold_end: {env.curriculum_tracker.threshold_end:.3f}")
    print(f"  current threshold: {env.curriculum_tracker.get_current_action_threshold():.3f}")

    if env.curriculum_tracker.threshold_start == 0.05:
        print("\n❌ PROBLEM: Environment using OLD threshold (0.05)!")
        print("   You changed Cell 3 but didn't restart runtime!")
        print("\n   FIX: Runtime → Restart runtime → Rerun all cells")
    elif env.curriculum_tracker.threshold_start == 0.01:
        print("\n✅ Environment using NEW threshold (0.01)")
        print("   Problem must be elsewhere - run full diagnostic")

# Test if environment can trade with forced signals
print("\n" + "="*80)
print("TESTING IF ENVIRONMENT CAN EXECUTE TRADES")
print("="*80)

obs, _ = env.reset()

# Force maximum signal strength
forced_action = np.ones(25, dtype=np.float32)
initial_trades = len(env.trades_this_episode)

obs, reward, done, truncated, info = env.step(forced_action)
trades_executed = len(env.trades_this_episode) - initial_trades

print(f"\nForced trade test (all signals = +1.0):")
print(f"  Trades executed: {trades_executed}")

if trades_executed == 0:
    print("\n  ❌ Environment CANNOT execute trades!")
    print("     Deeper blocking bug - run full diagnostic")
else:
    print(f"\n  ✅ Environment CAN trade! ({trades_executed} trades)")

    # Now test with SAC
    obs, _ = env.reset()
    action, _ = model.predict(obs, deterministic=False)
    threshold = env.curriculum_tracker.get_current_action_threshold()

    print(f"\nSAC signal test:")
    print(f"  Threshold: {threshold:.3f}")
    print(f"  SAC signals: min={action[:24].min():+.4f}, max={action[:24].max():+.4f}")
    print(f"  Signals >= threshold: {(np.abs(action[:24]) >= threshold).sum()}/24")

    if (np.abs(action[:24]) >= threshold).sum() == 0:
        print("\n  ❌ Even at 0.01, SAC signals too weak!")
        print("     Lower threshold to 0.005")

print("\n" + "="*80)

In [ ]:
### V42 Green-Up Verification ###

print("="*80)
print("🎯 V42 GREEN-UP STRATEGY VERIFICATION")
print("="*80)

# Get unwrapped environment
env = train_env
while hasattr(env, 'env'):
    env = env.env

# Check that green-up methods exist
print("\n✓ Checking V42 methods:")
print(f"  _check_in_play_transition: {hasattr(env, '_check_in_play_transition')}")
print(f"  _calculate_green_up_pnl: {hasattr(env, '_calculate_green_up_pnl')}")

# Check observation space (should still be 323)
print(f"\n✓ Observation space: {env.observation_space.shape}")
print(f"  Expected: (323,) - {'✅ CORRECT' if env.observation_space.shape == (323,) else '❌ ERROR'}")

# Check action space (should still be 25)
print(f"\n✓ Action space: {env.action_space.shape}")
print(f"  Expected: (25,) - {'✅ CORRECT' if env.action_space.shape == (25,) else '❌ ERROR'}")

# Check MTM/Sharpe accumulators
print(f"\n✓ V41 features maintained:")
print(f"  total_mtm_reward: {hasattr(env, 'total_mtm_reward')}")
print(f"  total_sharpe_reward: {hasattr(env, 'total_sharpe_reward')}")
print(f"  pnl_history: {hasattr(env, 'pnl_history')}")
print(f"  price_history: {hasattr(env, 'price_history')}")

# Run a quick episode to test green-up
print("\n" + "="*80)
print("TESTING GREEN-UP MECHANISM")
print("="*80)

obs, _ = env.reset()
episode_steps = 0
had_trades = False

for step in range(500):  # Enough steps to reach in-play
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    episode_steps += 1

    if info.get('trades_executed', 0) > 0:
        had_trades = True

    if done:
        print(f"\n✓ Episode completed (Green-up executed):")
        print(f"  Steps to in-play: {episode_steps}")
        print(f"  Trades executed: {len(env.trades_this_episode)}")
        print(f"  Final balance: ${env.balance:.2f}")
        print(f"  Final P&L: ${info['final_pnl']:.2f}")
        print(f"  Positions at end: {len(env.positions)} (should be 0)")

        if len(env.positions) == 0:
            print("\n✅ GREEN-UP WORKING: All positions closed!")
        else:
            print("\n⚠️  WARNING: Positions still open after green-up")

        break

if not done:
    print(f"\n⚠️  Episode didn't complete in {step} steps")
    print("  (May need more steps to reach in-play)")

print("\n" + "="*80)
print("🚀 V42 VERIFICATION COMPLETE")
print("="*80)
print("\nAll V41 features maintained: ✅")
print("Green-up strategy implemented: ✅")
print("Ready for training: ✅")
print("="*80)


In [ ]:
  ### PRE-TRAINING VERIFICATION - Run ONE Complete Episode ###

import numpy as np
import pandas as pd

print("="*80)
print("🔬 PRE-TRAINING VERIFICATION: Running ONE Complete Episode")
print("="*80)

print("\nThis will run exactly one episode to verify:")
print("  • Trades execute during actual training loop")
print("  • Metrics are collected correctly")
print("  • Rewards are calculated properly")
print("  • CSV logging works")
print("="*80)

# Get the unwrapped environment for inspection
test_env = train_env
base_env = test_env
while hasattr(base_env, 'env'):
    base_env = base_env.env

print(f"\n1️⃣ Environment Setup:")
print(f"   Type: {type(base_env).__name__}")
print(f"   Wrapper chain: {type(train_env).__name__} (should include Monitor)")

# Reset and run one episode
print(f"\n2️⃣ Running one complete episode...")
print("="*80)

obs, info = train_env.reset()
done = False
truncated = False
step_count = 0
total_reward = 0.0
trades_per_step = []

# Track rewards
step_rewards = []
mtm_rewards = []
sharpe_rewards = []

while not done and not truncated and step_count < 1000:
    # Get action from model
    action, _ = model.predict(obs, deterministic=False)

    # Execute step
    obs, reward, done, truncated, info = train_env.step(action)

    step_count += 1
    total_reward += reward
    step_rewards.append(reward)

    # Track trades this step
    current_trades = len(base_env.trades_this_episode)
    trades_per_step.append(current_trades)

    # Print progress every 100 steps
    if step_count % 100 == 0:
        print(f"   Step {step_count}: {current_trades} trades so far, reward={reward:+.2f}")

print(f"\n✅ Episode completed!")
print(f"   Total steps: {step_count}")
print(f"   Episode ended: done={done}, truncated={truncated}")

# Get final metrics
final_trades = len(base_env.trades_this_episode)
final_balance = base_env.balance
final_pnl = final_balance - base_env.initial_balance

print(f"\n3️⃣ Episode Metrics:")
print("="*80)
print(f"   Trades executed: {final_trades}")
print(f"   Final balance: ${final_balance:.2f}")
print(f"   Initial balance: ${base_env.initial_balance:.2f}")
print(f"   Realized P&L: ${final_pnl:+.2f}")
print(f"   Total reward: {total_reward:+.2f}")
print(f"   Avg reward/step: {total_reward/step_count:+.4f}")

# Check info dict
print(f"\n4️⃣ Info Dictionary Contents:")
print("="*80)
print(f"   Keys: {list(info.keys())}")

if 'episode' in info:
    print(f"   Episode info keys: {list(info['episode'].keys())}")
    print(f"\n   Episode info values:")
    for key, value in info['episode'].items():
        print(f"      {key}: {value}")

# Check no-trade penalty
if 'no_trade_streak' in info or ('episode' in info and 'no_trade_streak' in info['episode']):
    if 'episode' in info:
        streak = info['episode'].get('no_trade_streak', 0)
        penalty = info['episode'].get('no_trade_penalty', 0.0)
    else:
        streak = info.get('no_trade_streak', 0)
        penalty = info.get('no_trade_penalty', 0.0)

    print(f"\n5️⃣ No-Trade Penalty System:")
    print("="*80)
    print(f"   No-trade streak: {streak}")
    print(f"   Penalty applied: {penalty:.2f}")
    print(f"   Had trades: {final_trades > 0}")

    if final_trades > 0 and streak > 0:
        print(f"   ⚠️  WARNING: Had trades but streak > 0!")
    elif final_trades == 0 and streak == 0:
        print(f"   ⚠️  WARNING: No trades but streak = 0!")
    else:
        print(f"   ✅ Penalty system working correctly")

# Analyze trade distribution
if final_trades > 0:
    print(f"\n6️⃣ Trade Analysis:")
    print("="*80)

    trade_steps = [i for i in range(len(trades_per_step)) if i == 0 or trades_per_step[i] > trades_per_step[i-1]]

    print(f"   Total trades: {final_trades}")
    print(f"   Steps with trades: {len(trade_steps)}")
    print(f"   Trade rate: {len(trade_steps)/step_count*100:.1f}%")

    # Sample some trades
    if len(base_env.trades_this_episode) > 0:
        print(f"\n   First 3 trades:")
        for i, trade in enumerate(base_env.trades_this_episode[:3]):
            print(f"      Trade {i+1}: {trade['side']} runner {trade['runner']} @ {trade['price']:.2f}, stake=${trade['stake']:.2f}")

        if len(base_env.trades_this_episode) > 3:
            print(f"   ... and {len(base_env.trades_this_episode) - 3} more")

# Check what would be saved to CSV
print(f"\n7️⃣ What Would Be Saved to CSV:")
print("="*80)

# Simulate what TrainingMetricsCallback would save
if 'episode' in info:
    episode_info = info['episode']
    csv_data = {
        'Episode': 1,
        'Step': step_count,
        'Balance': final_balance,
        'Num_Trades': episode_info.get('num_trades', final_trades),
        'Realized_PnL': final_pnl,
        'No_Trade_Streak': episode_info.get('no_trade_streak', 0),
        'No_Trade_Penalty': episode_info.get('no_trade_penalty', 0.0),
        'Had_Trades': episode_info.get('had_trades', final_trades > 0),
        'Max_Drawdown': (base_env.peak_balance - final_balance) / base_env.peak_balance if base_env.peak_balance > 0 else 0.0,
        'MTM_Reward': info.get('total_mtm_reward', 0.0),
        'Sharpe_Reward': info.get('total_sharpe_reward', 0.0)
    }
else:
    csv_data = {
        'Episode': 1,
        'Step': step_count,
        'Balance': final_balance,
        'Num_Trades': final_trades,
        'Realized_PnL': final_pnl,
        'No_Trade_Streak': info.get('no_trade_streak', 0),
        'No_Trade_Penalty': info.get('no_trade_penalty', 0.0),
        'Had_Trades': final_trades > 0,
        'Max_Drawdown': (base_env.peak_balance - final_balance) / base_env.peak_balance if base_env.peak_balance > 0 else 0.0,
        'MTM_Reward': info.get('total_mtm_reward', 0.0),
        'Sharpe_Reward': info.get('total_sharpe_reward', 0.0)
    }

print("\n   CSV row that would be saved:")
for key, value in csv_data.items():
    print(f"      {key}: {value}")

# Final verdict
print(f"\n" + "="*80)
print("🎯 VERIFICATION RESULT")
print("="*80)

issues = []

if final_trades == 0:
    issues.append("❌ NO TRADES EXECUTED!")
    print("\n❌ CRITICAL ISSUE: No trades executed!")
    print("   DO NOT proceed to full training")
    print("   Run the COMPREHENSIVE_DIAGNOSTIC.py to find the blocking point")
else:
    print(f"\n✅ SUCCESS: {final_trades} trades executed!")

if final_balance == base_env.initial_balance and final_trades > 0:
    issues.append("⚠️  Balance unchanged despite trades")
    print("⚠️  WARNING: Balance unchanged despite trades")

if abs(total_reward) < 0.01:
    issues.append("⚠️  Total reward near zero")
    print("⚠️  WARNING: Total reward very small")

if final_trades == 0 and csv_data['No_Trade_Streak'] == 0:
    issues.append("⚠️  No-trade penalty not working")
    print("⚠️  WARNING: No-trade penalty system not working")

if len(issues) == 0:
    print("\n" + "="*80)
    print("🚀 ALL CHECKS PASSED - READY FOR FULL TRAINING!")
    print("="*80)
    print("\nExpected results in full training:")
    print(f"  • Episodes will execute {final_trades} ± 5 trades")
    print(f"  • Balance will vary around ${final_balance:.0f}")
    print(f"  • Rewards will average around {total_reward:.0f}")
    print("\nYou can proceed to Cell 6 (Training) with confidence!")
else:
    print("\n" + "="*80)
    print("⚠️  ISSUES DETECTED - DO NOT START FULL TRAINING YET")
    print("="*80)
    print("\nIssues found:")
    for issue in issues:
        print(f"  {issue}")
    print("\nResolve these issues before starting full training.")

print("\n" + "="*80)

In [ ]:
### DATA STRUCTURE DIAGNOSTIC ###

import pandas as pd
import random

# Load one race file
race_file = random.choice(train_files)
df = pd.read_parquet(race_file)

print("="*80)
print("DATA STRUCTURE DIAGNOSTIC")
print("="*80)

print(f"\n1️⃣ File: {race_file.split('/')[-1]}")
print(f"   Shape: {df.shape}")
print(f"   Columns: {len(df.columns)}")

print(f"\n2️⃣ First row columns (first 50):")
for i, col in enumerate(df.columns[:50]):
    print(f"   {i}: {col}")

print(f"\n3️⃣ Looking for runner data...")

# Check for runner columns
runner_cols = [col for col in df.columns if 'runner' in col.lower()]
print(f"   Found {len(runner_cols)} columns with 'runner' in name")

if len(runner_cols) > 0:
    print(f"\n   First 10 runner columns:")
    for col in runner_cols[:10]:
        print(f"      {col}")
else:
    print(f"   ❌ NO columns with 'runner' found!")

# Check specific columns we're looking for
print(f"\n4️⃣ Checking for expected columns:")
expected = [
    'runner_0_back_1',
    'runner_0_lay_1',
    'runner_0_back_vol_1',
    'runner_count'
]

for col in expected:
    exists = col in df.columns
    symbol = "✅" if exists else "❌"
    print(f"   {symbol} {col}: {exists}")

# Check first row values
print(f"\n5️⃣ First row sample data:")
row = df.iloc[0]

if 'runner_count' in df.columns:
    print(f"   runner_count: {row['runner_count']}")

# Try to get runner 0 data
prefix = 'runner_0_'
for suffix in ['back_1', 'lay_1', 'back_vol_1', 'lay_vol_1']:
    col_name = f'{prefix}{suffix}'
    if col_name in df.columns:
        print(f"   ✅ {col_name}: {row[col_name]}")
    else:
        print(f"   ❌ {col_name}: NOT FOUND (using default)")

print("\n" + "="*80)
# ```

# ---

# ## 🎯 **Expected Results**

# ### **If data structure is correct:**
# ```
# ✅ runner_0_back_1: True
# ✅ runner_0_lay_1: True
#    runner_0_back_1: 2.58
#    runner_0_lay_1: 2.60
# ```

# ### **If data structure is wrong (LIKELY):**
# ```
# ❌ runner_0_back_1: False
# ❌ runner_0_lay_1: False
#    runner_0_back_1: NOT FOUND (using default)

In [ ]:
### IMMEDIATE DIAGNOSTIC - Run this NOW to find blocking point ###

import sys
import numpy as np
import pandas as pd
import random

print("="*80)
print("🔬 EMERGENCY DIAGNOSTIC - Finding Why 0 Trades")
print("="*80)

# Get the unwrapped environment
test_env = train_env
while hasattr(test_env, 'env'):
    test_env = test_env.env

print(f"\nEnvironment type: {type(test_env).__name__}")

# Test 1: Check data structure
print("\n" + "="*80)
print("1️⃣ DATA STRUCTURE CHECK")
print("="*80)

try:
    # Load a random race
    race_file = random.choice(train_files)
    df = pd.read_parquet(race_file)
    row = df.iloc[0]

    print(f"\nRace file: {race_file.split('/')[-1]}")
    print(f"Shape: {df.shape}")
    print(f"runner_count: {row.get('runner_count', 'MISSING!')}")

    # Check column format
    print("\n📋 Checking column names:")

    runner_cols = [c for c in df.columns if 'run[' in c]
    if len(runner_cols) > 0:
        print(f"  ✅ Found {len(runner_cols)} columns with 'run[' format")
        print(f"\n  First 10 run[ columns:")
        for col in runner_cols[:10]:
            print(f"    {col}: {row[col]}")
    else:
        print(f"  ❌ NO columns with 'run[' format found!")

        # Check for alternative formats
        alt_cols = [c for c in df.columns if 'runner' in c.lower() or 'back' in c.lower()]
        print(f"\n  Found {len(alt_cols)} alternative columns:")
        for col in alt_cols[:10]:
            print(f"    {col}")

    # Test get_runner_data on this row
    print("\n📊 Testing get_runner_data() on row 0:")
    for runner_idx in range(min(3, int(row.get('runner_count', 0)))):
        runner_data = get_runner_data(row, runner_idx)
        if runner_data:
            print(f"  ✅ Runner {runner_idx}: back={runner_data['back_1']:.2f}, lay={runner_data['lay_1']:.2f}")
        else:
            print(f"  ❌ Runner {runner_idx}: FAILED")

except Exception as e:
    print(f"❌ Error: {str(e)}")

# Test 2: Reset environment and check
print("\n" + "="*80)
print("2️⃣ ENVIRONMENT RESET CHECK")
print("="*80)

try:
    obs, info = test_env.reset()

    print(f"\nAfter reset:")
    print(f"  runner_count: {test_env.runner_count}")
    print(f"  balance: ${test_env.balance:.2f}")
    print(f"  data shape: {test_env.current_race_df.shape}")
    print(f"  step_idx: {test_env.step_idx}")

    # Test runner data extraction
    print(f"\n📊 Testing runner data extraction:")
    current_row = test_env.current_race_df.iloc[0]

    valid_runners = 0
    for runner_idx in range(min(5, test_env.runner_count)):
        runner_data = get_runner_data(current_row, runner_idx)
        if runner_data:
            valid_runners += 1
            print(f"  ✅ Runner {runner_idx}: back={runner_data['back_1']:.2f}, lay={runner_data['lay_1']:.2f}")
        else:
            print(f"  ❌ Runner {runner_idx}: get_runner_data() returned None")

    if valid_runners == 0:
        print(f"\n❌ BLOCKING ISSUE: ALL runners return None!")
        print(f"   This is why 0 trades - no valid data!")
    else:
        print(f"\n✅ Data extraction working ({valid_runners}/{min(5, test_env.runner_count)} valid)")

except Exception as e:
    print(f"❌ Error: {str(e)}")
    import traceback
    traceback.print_exc()

# Test 3: Try to execute one step
print("\n" + "="*80)
print("3️⃣ SINGLE STEP EXECUTION TEST")
print("="*80)

try:
    obs, _ = test_env.reset()

    # Get curriculum values
    if hasattr(test_env, 'curriculum_tracker') and test_env.curriculum_tracker:
        threshold = test_env.curriculum_tracker.get_current_action_threshold()
        min_liability = test_env.curriculum_tracker.get_current_min_liability()
    else:
        threshold = 0.01
        min_liability = 0.05

    print(f"\nCurriculum settings:")
    print(f"  action_threshold: {threshold:.4f}")
    print(f"  min_liability: ${min_liability:.2f}")

    # Create a forced strong action
    forced_action = np.ones(25, dtype=np.float32)  # All signals = +1.0
    print(f"\nForced action: all signals = +1.0, allocation = +1.0")

    # Count blocking before step
    print(f"\nBefore step:")
    print(f"  trades_this_episode: {len(test_env.trades_this_episode)}")

    # Execute step
    obs, reward, done, truncated, info = test_env.step(forced_action)

    trades_executed = len(test_env.trades_this_episode)

    print(f"\nAfter step:")
    print(f"  trades_executed: {trades_executed}")
    print(f"  balance: ${test_env.balance:.2f}")
    print(f"  reward: {reward:.2f}")

    if trades_executed == 0:
        print(f"\n❌ CRITICAL: Even with forced max signals, NO trades!")
        print(f"   This proves there's a blocking condition")

        # Manually check what would block
        current_row = test_env.current_race_df.iloc[0]
        current_exposure = test_env._get_total_exposure_with_liability()
        unreserved_capital = max(0.0, test_env.balance - current_exposure)
        available_capital = unreserved_capital * (1.0 - RESERVE_RATIO)
        trade_budget = available_capital * 1.0 * MAX_EXPOSURE_MULTIPLIER

        print(f"\n   Capital calculation:")
        print(f"     balance: ${test_env.balance:.2f}")
        print(f"     exposure: ${current_exposure:.2f}")
        print(f"     unreserved: ${unreserved_capital:.2f}")
        print(f"     reserve_ratio: {RESERVE_RATIO*100:.0f}%")
        print(f"     available: ${available_capital:.2f}")
        print(f"     trade_budget: ${trade_budget:.2f}")

        if trade_budget < min_liability:
            print(f"\n   ❌ FOUND IT: Budget ${trade_budget:.2f} < min_liability ${min_liability:.2f}")
            print(f"      Even with max allocation, can't afford minimum trade!")

        # Check individual runners
        print(f"\n   Checking each runner:")
        blocked_data = 0
        blocked_price = 0
        blocked_budget = 0

        for runner_idx in range(min(test_env.runner_count, 5)):
            runner_data = get_runner_data(current_row, runner_idx)

            if runner_data is None:
                blocked_data += 1
                print(f"     Runner {runner_idx}: ❌ data=None")
            else:
                price = runner_data['back_1']
                if price < 1.01 or price > 1000:
                    blocked_price += 1
                    print(f"     Runner {runner_idx}: ❌ price={price:.2f} (out of range)")
                else:
                    signal_strength = 1.0  # Max signal
                    runner_budget = trade_budget * signal_strength

                    if runner_budget < min_liability:
                        blocked_budget += 1
                        print(f"     Runner {runner_idx}: ❌ budget=${runner_budget:.2f} < ${min_liability:.2f}")
                    else:
                        print(f"     Runner {runner_idx}: ✅ Should trade! (but didn't)")

        print(f"\n   SUMMARY:")
        print(f"     Blocked by data: {blocked_data}")
        print(f"     Blocked by price: {blocked_price}")
        print(f"     Blocked by budget: {blocked_budget}")
    else:
        print(f"\n✅ SUCCESS: {trades_executed} trades executed!")
        print(f"   System CAN trade - problem is with SAC signals")

except Exception as e:
    print(f"❌ Error: {str(e)}")
    import traceback
    traceback.print_exc()

# Test 4: Check SAC action values
print("\n" + "="*80)
print("4️⃣ SAC ACTION VALUES CHECK")
print("="*80)

try:
    obs, _ = test_env.reset()

    # Get SAC action
    action, _ = model.predict(obs, deterministic=False)

    runner_signals = action[:24]
    allocation = action[24]

    print(f"\nSAC action analysis:")
    print(f"  signals: min={runner_signals.min():+.4f}, max={runner_signals.max():+.4f}")
    print(f"  signals mean: {np.mean(np.abs(runner_signals)):.4f}")
    print(f"  allocation: {allocation:+.4f}")

    # Check against threshold
    if hasattr(test_env, 'curriculum_tracker') and test_env.curriculum_tracker:
        threshold = test_env.curriculum_tracker.get_current_action_threshold()
    else:
        threshold = 0.01

    strong_signals = (np.abs(runner_signals) >= threshold).sum()
    print(f"\n  Signals >= threshold {threshold:.4f}: {strong_signals}/24")

    if strong_signals == 0:
        print(f"\n  ❌ ALL SAC signals too weak!")
        print(f"     Even at threshold {threshold:.4f}, nothing passes")
        print(f"     This would cause 0 trades")
    else:
        print(f"\n  ✅ {strong_signals} signals pass threshold")

except Exception as e:
    print(f"❌ Error: {str(e)}")

print("\n" + "="*80)
print("DIAGNOSTIC COMPLETE")
print("="*80)

print("\n🎯 MOST LIKELY ISSUE:")
print("   Check the output above to see which test failed!")
print("\n   If Test 1 shows 'NO columns with run[ format':")
print("     → Column names don't match - need to update get_runner_data()")
print("\n   If Test 2 shows 'ALL runners return None':")
print("     → Data extraction broken - check column format")
print("\n   If Test 3 shows 'Budget < min_liability':")
print("     → Lower INITIAL_MIN_LIABILITY or increase capital")
print("\n   If Test 3 shows 'ALL SAC signals too weak':")
print("     → Lower INITIAL_ACTION_THRESHOLD")

In [ ]:
### 🔬 DIAGNOSTIC CELL - V41 BUG DETECTION ###
# Drop this into your notebook to identify all critical bugs
# Run AFTER Cell 5 (environment setup) but BEFORE training

import numpy as np
import pandas as pd
from copy import deepcopy

print("="*80)
print("🔬 V41 COMPREHENSIVE BUG DIAGNOSTIC")
print("="*80)
print("\nThis will test:")
print("  1. MTM Reward Calculation")
print("  2. Sharpe Reward Calculation")
print("  3. Breakeven/Balance Reset Bug")
print("  4. Validation Callback Issues")
print("\n" + "="*80)

# ============================================================================
# TEST 1: MTM REWARD CALCULATION
# ============================================================================

print("\n" + "="*80)
print("TEST 1: MTM REWARD CALCULATION")
print("="*80)

# Get the actual environment
test_env = train_env
while hasattr(test_env, 'env'):
    test_env = test_env.env

# Reset and take a few steps
obs, info = test_env.reset()
initial_balance = test_env.balance

print(f"\nInitial balance: ${initial_balance:.2f}")
print(f"Testing 10 steps to see if MTM rewards are calculated...\n")

mtm_rewards = []
balances = []
step_count = 0

for i in range(10):
    # Generate a random action
    action = np.random.randn(25).astype(np.float32)

    # Store balance before step
    balance_before = test_env.balance

    # Take step
    obs, reward, done, truncated, info = test_env.step(action)
    step_count += 1

    # Check if environment has MTM reward attribute/calculation
    mtm_reward = 0.0

    # Try to find MTM reward in different places
    if hasattr(test_env, 'mtm_reward'):
        mtm_reward = test_env.mtm_reward
    elif hasattr(test_env, 'last_mtm_reward'):
        mtm_reward = test_env.last_mtm_reward
    elif 'mtm_reward' in info:
        mtm_reward = info['mtm_reward']

    mtm_rewards.append(mtm_reward)
    balances.append(test_env.balance)

    balance_change = test_env.balance - balance_before

    print(f"Step {i+1}: Balance=${test_env.balance:8.2f} (Δ${balance_change:+8.2f}) | "
          f"Reward={reward:+8.4f} | MTM={mtm_reward:+8.4f}")

    if done or truncated:
        print(f"  Episode ended at step {i+1}")
        break

print(f"\nResults:")
print(f"  Total steps: {step_count}")
print(f"  Non-zero MTM rewards: {sum(1 for r in mtm_rewards if abs(r) > 1e-6)}/{len(mtm_rewards)}")
print(f"  MTM reward range: [{min(mtm_rewards):.4f}, {max(mtm_rewards):.4f}]")
print(f"  Balance changed: {abs(test_env.balance - initial_balance) > 0.01}")

# Diagnosis
if sum(1 for r in mtm_rewards if abs(r) > 1e-6) == 0:
    print(f"\n❌ BUG DETECTED: MTM rewards are ALL ZERO!")
    print(f"   Expected: Non-zero values reflecting position value changes")
    print(f"   Actual: All zeros")
    print(f"\n   🔍 Check in MarketMakingEnv.step():")
    print(f"      - Is mtm_reward being calculated?")
    print(f"      - Is it being added to the reward?")
    print(f"      - Search for 'mtm' or 'mark_to_market' in the code")
elif sum(1 for r in mtm_rewards if abs(r) > 1e-6) < len(mtm_rewards) * 0.5:
    print(f"\n⚠️  WARNING: MTM rewards mostly zero ({sum(1 for r in mtm_rewards if abs(r) > 1e-6)}/{len(mtm_rewards)} non-zero)")
    print(f"   This might be expected if no positions are open")
else:
    print(f"\n✅ MTM rewards appear to be working!")
    print(f"   {sum(1 for r in mtm_rewards if abs(r) > 1e-6)}/{len(mtm_rewards)} steps had non-zero MTM rewards")

# ============================================================================
# TEST 2: SHARPE REWARD CALCULATION
# ============================================================================

print("\n" + "="*80)
print("TEST 2: SHARPE REWARD CALCULATION")
print("="*80)

# Reset environment
obs, info = test_env.reset()

print(f"\nTesting Sharpe reward calculation over full episode...\n")

sharpe_rewards = []
episode_returns = []
step_count = 0

for i in range(1000):  # Max 1000 steps
    action = np.random.randn(25).astype(np.float32)
    obs, reward, done, truncated, info = test_env.step(action)
    step_count += 1

    # Try to find Sharpe reward
    sharpe_reward = 0.0
    if hasattr(test_env, 'sharpe_reward'):
        sharpe_reward = test_env.sharpe_reward
    elif hasattr(test_env, 'last_sharpe_reward'):
        sharpe_reward = test_env.last_sharpe_reward
    elif 'sharpe_reward' in info:
        sharpe_reward = info['sharpe_reward']

    sharpe_rewards.append(sharpe_reward)
    episode_returns.append(reward)

    if done or truncated:
        break

print(f"Results:")
print(f"  Episode steps: {step_count}")
print(f"  Non-zero Sharpe rewards: {sum(1 for r in sharpe_rewards if abs(r) > 1e-6)}/{len(sharpe_rewards)}")
print(f"  Sharpe reward range: [{min(sharpe_rewards):.6f}, {max(sharpe_rewards):.6f}]")

# Calculate what Sharpe SHOULD be
if len(episode_returns) > 10:
    expected_sharpe = np.mean(episode_returns) / (np.std(episode_returns) + 1e-8)
    print(f"  Expected Sharpe ratio: {expected_sharpe:.6f}")
else:
    expected_sharpe = None
    print(f"  (Not enough returns to calculate expected Sharpe)")

# Diagnosis
if sum(1 for r in sharpe_rewards if abs(r) > 1e-6) == 0:
    print(f"\n❌ BUG DETECTED: Sharpe rewards are ALL ZERO!")
    print(f"   Expected: Non-zero values based on return distribution")
    if expected_sharpe:
        print(f"   Expected Sharpe: ~{expected_sharpe:.6f}")
    print(f"   Actual: All zeros")
    print(f"\n   🔍 Check in MarketMakingEnv.step():")
    print(f"      - Is sharpe_reward being calculated?")
    print(f"      - Is it using episode returns history?")
    print(f"      - Search for 'sharpe' in the code")
elif sum(1 for r in sharpe_rewards if abs(r) > 1e-6) < 10:
    print(f"\n⚠️  WARNING: Very few non-zero Sharpe rewards")
    print(f"   Sharpe calculation might require more data")
else:
    print(f"\n✅ Sharpe rewards appear to be working!")

# ============================================================================
# TEST 3: BREAKEVEN / BALANCE RESET BUG
# ============================================================================

print("\n" + "="*80)
print("TEST 3: BREAKEVEN / BALANCE RESET BUG")
print("="*80)

print(f"\nRunning 5 complete episodes to check balance persistence...\n")

episode_data = []

for ep in range(5):
    obs, info = test_env.reset()

    starting_balance = test_env.balance

    # Run episode
    total_reward = 0
    trades_executed = 0

    for step in range(1000):
        action = np.random.randn(25).astype(np.float32)
        obs, reward, done, truncated, info = test_env.step(action)
        total_reward += reward

        # Count trades
        if hasattr(test_env, 'trades_this_episode'):
            trades_executed = len(test_env.trades_this_episode)

        if done or truncated:
            break

    final_balance = test_env.balance
    pnl = final_balance - starting_balance

    episode_data.append({
        'episode': ep + 1,
        'start_balance': starting_balance,
        'final_balance': final_balance,
        'pnl': pnl,
        'trades': trades_executed,
        'total_reward': total_reward
    })

    print(f"Episode {ep+1}: Start=${starting_balance:8.2f} → End=${final_balance:8.2f} | "
          f"P&L=${pnl:+8.2f} | Trades={trades_executed}")

# Analysis
print(f"\nAnalysis:")

# Check if balances reset to 1000
resets_to_1000 = sum(1 for e in episode_data if abs(e['final_balance'] - 1000.0) < 0.01)
print(f"  Episodes ending at exactly $1,000: {resets_to_1000}/5")

# Check if next episode starts with previous episode's final balance
balance_carries_forward = True
for i in range(1, len(episode_data)):
    prev_final = episode_data[i-1]['final_balance']
    curr_start = episode_data[i]['start_balance']
    if abs(prev_final - curr_start) > 0.01:
        balance_carries_forward = False
        print(f"  Episode {i+1} started with ${curr_start:.2f} but Episode {i} ended with ${prev_final:.2f}")
        break

# Check for breakeven with trades
breakeven_with_trades = [e for e in episode_data if abs(e['pnl']) < 0.01 and e['trades'] > 10]

# Diagnosis
if resets_to_1000 >= 3:
    print(f"\n❌ BUG DETECTED: BREAKEVEN RESET BUG!")
    print(f"   {resets_to_1000}/5 episodes ended at exactly $1,000")
    print(f"   This suggests balance is being reset instead of calculated")
    print(f"\n   🔍 Check in MarketMakingEnv.reset():")
    print(f"      - Does reset() set self.balance = 1000?")
    print(f"      - Should it carry forward from previous episode?")
    print(f"      - Check position settlement logic")

if not balance_carries_forward:
    print(f"\n❌ BUG DETECTED: Balance NOT carrying forward between episodes!")
    print(f"   Each episode resets to a fixed value")
    print(f"\n   🔍 This explains the total P&L mismatch!")
    print(f"      - Episodes should start where previous ended")
    print(f"      - Check MarketMakingEnv.reset() logic")

if len(breakeven_with_trades) > 0:
    print(f"\n⚠️  WARNING: Found {len(breakeven_with_trades)} episodes with >10 trades but $0 P&L")
    for e in breakeven_with_trades:
        print(f"      Episode {e['episode']}: {e['trades']} trades → ${e['pnl']:.2f} P&L")
    print(f"   This is highly suspicious - check position settlement")

if resets_to_1000 < 2 and balance_carries_forward:
    print(f"\n✅ Balance reset appears to be working correctly")
    print(f"   Balances carry forward between episodes")

# ============================================================================
# TEST 4: VALIDATION CALLBACK ISSUES
# ============================================================================

print("\n" + "="*80)
print("TEST 4: VALIDATION CALLBACK ISSUES")
print("="*80)

print(f"\nTesting validation environment access...\n")

# Check if validation environment exists
if 'val_env' not in globals():
    print(f"❌ ERROR: val_env not found!")
    print(f"   Validation environment wasn't created")
else:
    print(f"✅ Validation environment exists")

    # Check if it's wrapped
    print(f"\nChecking environment wrapper structure:")

    current_env = val_env
    wrapper_chain = []
    depth = 0

    while True:
        wrapper_chain.append(type(current_env).__name__)
        print(f"  Level {depth}: {type(current_env).__name__}")

        if hasattr(current_env, 'env'):
            current_env = current_env.env
            depth += 1
        else:
            break

    # Test direct access
    print(f"\nTesting direct attribute access on val_env:")

    test_attributes = [
        'trades_this_episode',
        'balance',
        'current_race_df',
        'step_count'
    ]

    direct_access_works = {}

    for attr in test_attributes:
        try:
            value = getattr(val_env, attr, None)
            direct_access_works[attr] = value is not None
            print(f"  val_env.{attr}: {'✅ Accessible' if value is not None else '⚠️  Returns None'}")
        except AttributeError as e:
            direct_access_works[attr] = False
            print(f"  val_env.{attr}: ❌ AttributeError")

    # Test unwrapped access
    print(f"\nTesting unwrapped attribute access:")

    unwrapped_env = val_env
    while hasattr(unwrapped_env, 'env'):
        unwrapped_env = unwrapped_env.env

    unwrapped_access_works = {}

    for attr in test_attributes:
        try:
            value = getattr(unwrapped_env, attr, None)
            unwrapped_access_works[attr] = value is not None
            print(f"  unwrapped_env.{attr}: {'✅ Accessible' if value is not None else '⚠️  Returns None'}")
        except AttributeError as e:
            unwrapped_access_works[attr] = False
            print(f"  unwrapped_env.{attr}: ❌ AttributeError")

    # Diagnosis
    print(f"\nDiagnosis:")

    if 'Monitor' in wrapper_chain:
        print(f"  ✅ Validation environment IS wrapped in Monitor (as expected)")

    critical_attrs = ['trades_this_episode', 'balance']
    direct_fails = [attr for attr in critical_attrs if not direct_access_works.get(attr, False)]
    unwrapped_works = [attr for attr in critical_attrs if unwrapped_access_works.get(attr, False)]

    if len(direct_fails) > 0 and len(unwrapped_works) > 0:
        print(f"\n❌ BUG DETECTED: VALIDATION CALLBACK WRAPPER ISSUE!")
        print(f"   Direct access fails for: {direct_fails}")
        print(f"   But unwrapped access works for: {unwrapped_works}")
        print(f"\n   🔍 Fix in ValidationCallback:")
        print(f"      Add unwrapping logic:")
        print(f"      ```python")
        print(f"      env = self.val_env")
        print(f"      while hasattr(env, 'env'):")
        print(f"          env = env.env")
        print(f"      trades = env.trades_this_episode")
        print(f"      ```")
    elif len(unwrapped_works) == len(critical_attrs):
        print(f"  ✅ Unwrapped access works for all critical attributes")
        print(f"  ⚠️  But ValidationCallback needs to unwrap to access them")
    else:
        print(f"  ⚠️  Some attributes not accessible even when unwrapped")
        print(f"     This might be expected before reset() is called")

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*80)
print("📊 DIAGNOSTIC SUMMARY")
print("="*80)

print(f"\n🔍 Bugs Found:")

bugs_found = []

# MTM check
if sum(1 for r in mtm_rewards if abs(r) > 1e-6) == 0:
    bugs_found.append("❌ MTM Rewards: ALL ZERO (0% working)")

# Sharpe check
if sum(1 for r in sharpe_rewards if abs(r) > 1e-6) == 0:
    bugs_found.append("❌ Sharpe Rewards: ALL ZERO (0% working)")

# Breakeven check
if resets_to_1000 >= 3:
    bugs_found.append(f"❌ Breakeven Reset: {resets_to_1000}/5 episodes reset to $1,000")

# Validation check
if len(direct_fails) > 0 and len(unwrapped_works) > 0:
    bugs_found.append("❌ Validation Callback: Monitor unwrapping needed")

if len(bugs_found) == 0:
    print(f"\n✅ NO CRITICAL BUGS DETECTED!")
    print(f"   All systems appear to be working correctly")
else:
    for bug in bugs_found:
        print(f"  {bug}")

    print(f"\n⚠️  TOTAL BUGS FOUND: {len(bugs_found)}")
    print(f"\n🔧 RECOMMENDED ACTIONS:")
    print(f"   1. Fix each bug listed above")
    print(f"   2. Re-run this diagnostic cell")
    print(f"   3. Verify all tests pass")
    print(f"   4. THEN start training")

print("\n" + "="*80)
print("🔬 DIAGNOSTIC COMPLETE")
print("="*80)

In [ ]:
# ============================================================================
# DATA STRUCTURE DIAGNOSTIC - Run this in Colab
# ============================================================================
#
# PURPOSE: Check what columns your data actually has
# This will tell us which fix to apply
#
# INSTRUCTIONS:
# 1. In Colab, create a new cell after Cell 4 (data loading)
# 2. Copy-paste this entire code
# 3. Run the cell
# 4. Share the output to get the right fix
#
# ============================================================================

print("="*80)
print("🔍 DATA STRUCTURE DIAGNOSTIC")
print("="*80)

# Load one sample file
import pandas as pd

if len(train_files) == 0:
    print("\n❌ ERROR: No training files loaded!")
    print("   Make sure Cell 4 (data loading) ran successfully")
else:
    print(f"\n✓ Found {len(train_files)} training files")

    # Load first file
    sample_file = train_files[0]
    print(f"\n📂 Analyzing: {sample_file.split('/')[-1]}")

    try:
        df = pd.read_parquet(sample_file)
        print(f"✓ Loaded {len(df)} rows")

        print("\n" + "="*80)
        print("CHECKING RUNNER 0 COLUMNS")
        print("="*80)

        # Get all columns for runner 0
        runner0_cols = [col for col in df.columns if col.startswith('run[0].')]

        if len(runner0_cols) == 0:
            print("\n❌ No run[0]. columns found!")
            print("   Your data might use a different format")
            print("\n   Available columns sample:")
            for col in list(df.columns)[:20]:
                print(f"     {col}")
        else:
            print(f"\n✓ Found {len(runner0_cols)} columns for runner 0")

            # Check for critical columns
            critical_checks = {
                'Prices': ['run[0].back_price_1', 'run[0].lay_price_1'],
                'Volume L1': ['run[0].back_vol_1', 'run[0].lay_vol_1'],
                'Volume L2': ['run[0].back_vol_2', 'run[0].lay_vol_2'],
                'Volume L3': ['run[0].back_vol_3', 'run[0].lay_vol_3'],
                'Last Traded': ['run[0].ltp', 'run[0].ltv'],
                'Total Matched': ['run[0].total_matched'],
            }

            print("\n" + "="*80)
            print("CRITICAL FEATURES CHECK")
            print("="*80)

            results = {}
            for category, cols in critical_checks.items():
                has_all = all(col in df.columns for col in cols)
                results[category] = has_all

                status = "✓" if has_all else "❌"
                print(f"\n{status} {category}:")
                for col in cols:
                    exists = "✓" if col in df.columns else "❌"
                    print(f"   {exists} {col}")

            # Determine which solution to use
            print("\n" + "="*80)
            print("📊 SOLUTION RECOMMENDATION")
            print("="*80)

            has_prices = results['Prices']
            has_vol1 = results['Volume L1']
            has_vol2 = results['Volume L2']
            has_vol3 = results['Volume L3']

            if has_prices and has_vol1 and has_vol2 and has_vol3:
                print("\n✅ SOLUTION A: EXPAND get_runner_data()")
                print("   Your data HAS all depth levels (vol_1, vol_2, vol_3)")
                print("   I'll create a Cell 5 fix to extract all levels")
                solution = "A"

            elif has_prices and has_vol1:
                print("\n⚠️  SOLUTION C: PAD WITH ZEROS")
                print("   Your data only has vol_1 (no vol_2, vol_3)")
                print("   I'll create a Cell 7 fix to pad missing features")
                solution = "C"

            else:
                print("\n❌ CRITICAL: Missing essential columns")
                print("   Cannot determine solution without price/volume data")
                solution = "UNKNOWN"

            # Show full column list
            print("\n" + "="*80)
            print(f"ALL RUNNER 0 COLUMNS ({len(runner0_cols)})")
            print("="*80)

            for col in sorted(runner0_cols):
                print(f"  {col}")

            # Summary
            print("\n" + "="*80)
            print("📋 SUMMARY")
            print("="*80)

            print(f"\nData file: {sample_file.split('/')[-1]}")
            print(f"Rows: {len(df)}")
            print(f"Total columns: {len(df.columns)}")
            print(f"Runner 0 columns: {len(runner0_cols)}")
            print(f"\nRecommended solution: {solution}")

            if solution == "A":
                print("\n✅ Next step: Apply Cell 5 fix (expand data extraction)")
            elif solution == "C":
                print("\n✅ Next step: Apply Cell 7 fix (pad with zeros)")
            else:
                print("\n❌ Need to investigate data structure further")

    except Exception as e:
        print(f"\n❌ ERROR loading file: {e}")
        print("\n   Try checking if the file path is correct")

print("\n" + "="*80)

In [ ]:
import pandas as pd

# Load a sample file
sample_file = '/content/drive/MyDrive/Betfair_RL/race_out/2026-01-12_Gawler_Race8_1.252604001.parquet'
df = pd.read_parquet(sample_file)

print("Testing get_runner_data() with corrected column names:")
print("="*60)

# Get first row
row = df.iloc[0]

# Test runner 0
runner_data = get_runner_data(row, 0)

if runner_data:
    print("\n✅ SUCCESS: Data extracted correctly!")
    print(f"\nRunner 0 data:")
    for key, value in runner_data.items():
        print(f"  {key:15s}: {value:.4f}")
else:
    print("\n❌ FAILED: Could not extract runner data")

print("\n" + "="*60)

In [ ]:
race_file = getattr(env, 'current_race_file', None)
if race_file:
    print(f"  Race: {race_file.split('/')[-1]}")
else:
    print(f"  Race: (file path not stored)")

In [ ]:
### CELL 6 - TRAIN MODEL ###

print("="*60)
print("🚀 Starting V42 training run (Green-Up Strategy)")
print("="*60)
print(f"Training for {CURRICULUM_TOTAL_STEPS:,} steps")
print(f"Expected duration: ~4-6 hours")
print(f"Output directory: {BASE_PATH}")
print("="*60)

model.learn(
    total_timesteps=CURRICULUM_TOTAL_STEPS,
    callback=callbacks,
    progress_bar=False
)

# Save final model
final_model_path = f"{BASE_PATH}/final_model"
model.save(final_model_path)

print(f"\n{'='*60}")
print(f"✅ Training complete!")
print(f"💾 Final model saved: {final_model_path}")
print(f"📊 Metrics saved to: {BASE_PATH}/")
print(f"{'='*60}\n")